Armando Alberto Lluen Gallardo

Ejercicio 1

los repositorios de la clase revisados para hacer este primer ejercicio son:

Suavizado Add-one: https://github.com/kapumota/Actividades-CC0C2/blob/main/Cuadernos-CC0C2/Clase2/Modelos-lenguaje1.ipynb

Suavizado Add-k: https://github.com/kapumota/Actividades-CC0C2/blob/main/Cuadernos-CC0C2/Clase2/Modelos-lenguaje2.ipynb

Suavizado Backoff:  https://github.com/kapumota/Actividades-CC0C2/blob/main/Cuadernos-CC0C2/Clase2/Counts-backoff-suavizado.ipynb

Suavizado Stupid Backoff: https://github.com/kapumota/Actividades-CC0C2/blob/main/Cuadernos-CC0C2/Clase2/Topicos-avanzados.ipynb

Glove y word2vec: https://github.com/kapumota/Actividades-CC0C2/blob/main/Cuadernos-CC0C2/Clase4/Embeddings.ipynb

Repositorio de mi resolucion de la PC1: https://github.com/Kinartb/CC0C2/blob/main/PC1/pc1_nlp.py

Primero construimos el corpus con las oraciones que nos dicta en el enunciado del ejercicio, posteriormente utilizamos nltk para la tokenizacion e imprimimos el corpus para una visualiazacion preliminar.

In [1]:
import collections
from typing import List, Dict, Tuple
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Vocabulario
vocabulario = {'<s>', '</s>', 'a', 'all', 'model', 'models', 'some', 'useful', 'wrong'}

#corpus
frase_1 =  "all models are wrong"
frase_2 =  "a model is wrong"
frase_3 =  "some useful models are wrong"
lista_frases = [[frase_1], [frase_2], [frase_3]]

def construir_corpus(lista_frases: List[str]) -> List[str]:
    """
    Constructs a corpus as a flat list of tokens from a list of sentences.
    """
    corpus = []
    for frase in lista_frases:
        sentence = frase[0]
        tokens = word_tokenize(sentence)
        corpus.extend(tokens) # Extend corpus with tokens of this sentence
    return corpus

corpus = construir_corpus(lista_frases)
print(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['all', 'models', 'are', 'wrong', 'a', 'model', 'is', 'wrong', 'some', 'useful', 'models', 'are', 'wrong']


#Modelo N-Gramas

Primero implemenetamos el modelo de Ngrama sin usar ningun tipo de suavizado

Para ello implementamos la clase ModeloGrama cuyos valores de entrada sera el valor n, donde n es la cantidad de gramas (1, 2 o 3). Esta clase centa con atribuos como la cuenta de la aparicion de los ngramas, la cuenta del contexto de los ngramas y ademas del total de gramas que han aparecido.

Tiene la funcion 'Entrenar' cuyo paramentro de entrada es el corpus y lo que hace es recorrer todo el corpus para poder contar adecuadamente cada n-grama.

Tiene la funcion 'calcular_probabilidad' que con los atributos que ya se han entrenado con el corpus calcule la probabilidad de todos los bigramas

In [ ]:
#Calcular las probabilidades de todos los bigramas sin suavizado
class ModeloNGrama:
    def __init__(self, n:int):
        self.n = n
        self.cuenta_ngrama = collections.Counter()
        self.cuenta_contexto = collections.Counter()
        self.vocabulario = vocabulario = {'<s>', '</s>', 'a', 'all', 'model', 'models', 'some', 'useful', 'wrong'}
        self.total_ngrama = 0

    def entrenar(self, corpus: List[str]):
        for i in range(len(corpus) - self.n + 1):
            if corpus[i] not in self.vocabulario:
                self.vocabulario.add(corpus[i])
            ngrama = tuple(corpus[i:i + self.n])
            contexto = tuple(corpus[i:i + self.n - 1])
            self.cuenta_ngrama[ngrama] += 1
            self.cuenta_contexto[contexto] += 1
            self.total_ngrama += 1

    def calcular_probabilidad(self):
        print("Probabilidades de bigramas")
        for ngrama, cuenta in self.cuenta_ngrama.items():
            contexto = ngrama[:-1]
            probabilidad = cuenta / self.cuenta_contexto[contexto]
            print(f"P({ngrama[-1]} | {contexto[0]}) = {probabilidad:.4f}")

In [ ]:
modelo_bigrama = ModeloNGrama(n=2)
modelo_bigrama.entrenar(corpus=corpus)
modelo_bigrama.calcular_probabilidad() #Muestra las probabilidades

Probabilidades de bigramas
P(models | all) = 1.0000
P(are | models) = 1.0000
P(wrong | are) = 1.0000
P(a | wrong) = 0.5000
P(model | a) = 1.0000
P(is | model) = 1.0000
P(wrong | is) = 1.0000
P(some | wrong) = 0.5000
P(useful | some) = 1.0000
P(models | useful) = 1.0000


Este resultado muestra las probabilidades de bigramas en un modelo de lenguaje basado en N-gramas (en este caso, bigramas). Los bigramas son pares consecutivos de palabras, y la probabilidad de un bigrama indica la probabilidad de que una palabra aparezca después de otra en el contexto del corpus utilizado.

Cada línea de tu resultado representa una probabilidad condicional de que una palabra siga a otra palabra específica. En este caso, se calcula
P(palabra actual|palabra anterior), que es la probabilidad de que la "palabra actual" ocurra después de la "palabra anterior". Estas probabilidades se basan en la frecuencia con la que esos pares de palabras ocurren en el corpus.

Probabilidades de 1.0 (o 100%): Indican una relación fija en el corpus. Esto suele pasar si el corpus es pequeño o tiene patrones repetitivos; cada vez que aparece una palabra, la siguiente palabra es la misma.

Probabilidades menores a 1.0 (como 0.5): Sugieren que hay más de una posible palabra que puede seguir a la anterior en el corpus. En este caso, "wrong" no siempre es seguido por "a" ni por "some", sino que hay variabilidad.

#Modelo de lenguaje suavizado con AddOne (Laplace)
Hacemos una clase que representa un modelo de N-gramas con suavizado de Laplace. Esta clase tiene métodos para entrenar el modelo y calcular probabilidades de bigramas suavizadas.

Los atributos de esta clase es:
- n almacena el valor de n (para bigramas, n=2).
- bigram_counts y self.unigram_counts son diccionarios que almacenan el conteo de cada bigrama y unigram, respectivamente.
-vocabulario es un conjunto que contiene todas las palabras en el vocabulario.
-total_bigrams almacena el total de bigramas en el corpus, aunque no se utiliza en este código.

Tiene el metodo ´entrenar´ cuya entrada es el corpus realiza el conteo de bigramas y unigrams recorriendo las palabras en el corpus para formar pares consecutivos (bigramas) y también para contar cada palabra individual (unigramas).

Tiene el método "Calcular_probabilidades_suavizadas" en donde se usa el calculo de probabilidades segun el repositorio de la clase y se va imprimiendo bigrama por bigrama.

Tambien tiene el metodo "calcular_probabilidaes_bigrama" cuya entrada es el contexto y la palabra para calcular la probabilidad de un bigrama en especifico

In [ ]:
#Modelo con suavizado AddOne
class ModeloNGrama_AddOne:
    def __init__(self, n: int):
        self.n = n  # cuando es bigrama, n=2
        self.bigram_counts = {}
        self.unigram_counts = {}
        self.vocabulario = {'<s>', '</s>', 'a', 'all', 'model', 'models', 'some', 'useful', 'wrong'}
        self.total_bigrams = 0

    def entrenar(self, corpus: List[str]):
        # Conteo de bigramas y unigrams
        for i in range(len(corpus) - 1):
            bigram = (corpus[i], corpus[i + 1])
            unigram = corpus[i]

            # Conteo de bigramas
            if bigram in self.bigram_counts:
                self.bigram_counts[bigram] += 1
            else:
                self.bigram_counts[bigram] = 1
            # Conteo de unigrams
            if unigram in self.unigram_counts:
                self.unigram_counts[unigram] += 1
            else:
                self.unigram_counts[unigram] = 1

        # Contar el último unigram
        last_word = corpus[-1]
        if last_word in self.unigram_counts:
            self.unigram_counts[last_word] += 1
        else:
            self.unigram_counts[last_word] = 1

    def calcular_probabilidades_suavizadas(self):
        # Tamaño del vocabulario
        V = len(self.unigram_counts)
        laplace_probabilities = {}

        print("Probabilidades de bigramas (con suavizado Laplace)")
        for bigram, bigram_count in self.bigram_counts.items():
            w_n_1 = bigram[0]
            # Aplicando la ecuación P_Laplace(w_n | w_n-1) = (C(w_n-1 w_n) + 1) / (C(w_n-1) + V)
            laplace_prob = (bigram_count + 1) / (self.unigram_counts[w_n_1] + V)
            laplace_probabilities[bigram] = laplace_prob
            print(f"P({bigram[1]} | {bigram[0]}) = {laplace_prob:.4f}")

        return laplace_probabilities

    def calcular_probabilidad_bigrama(self, contexto: str, palabra: str):
        V = len(self.unigram_counts)
        bigram = (contexto, palabra)

        # Aplicando la ecuación para bigramas no vistos con suavizado Laplace
        cuenta_bigrama = self.bigram_counts.get(bigram, 0) + 1
        cuenta_contexto = self.unigram_counts.get(contexto, 0) + V
        probabilidad = cuenta_bigrama / cuenta_contexto
        print("\nLa probabilidad de el bigrama ingresado es")
        print(f"P({palabra} | {contexto}) = {probabilidad:.4f}")
        return probabilidad


In [ ]:
modelo_bigrama = ModeloNGrama_AddOne(2)
modelo_bigrama.entrenar(corpus)
modelo_bigrama.calcular_probabilidades_suavizadas()

# Calcular probabilidad de un bigrama específico
modelo_bigrama.calcular_probabilidad_bigrama('a', 'models')

Probabilidades de bigramas (con suavizado Laplace)
P(models | all) = 0.2000
P(are | models) = 0.2727
P(wrong | are) = 0.2727
P(a | wrong) = 0.1667
P(model | a) = 0.2000
P(is | model) = 0.2000
P(wrong | is) = 0.2000
P(some | wrong) = 0.1667
P(useful | some) = 0.2000
P(models | useful) = 0.2000

La probabilidad de el bigrama ingresado es
P(models | a) = 0.1000


0.1

Estos resultados muestran las probabilidades de bigramas calculadas utilizando el suavizado de Laplace (Add-One) en un modelo de bigramas. El suavizado de Laplace ayuda a asignar probabilidades a bigramas que podrían no aparecer en el corpus, evitando que su probabilidad sea cero.

#Modelo con Suavizado Add-k

Hacemos una clase que representa un modelo de N-gramas con suavizado Add-k. Esta clase tiene métodos para entrenar el modelo y calcular probabilidades de bigramas suavizadas.

Tiene como atributos:

- n: Especifica el valor de n para el modelo de N-gramas (en este caso, bigramas con n=2).

- k: El parámetro de suavizado Add-k, que se puede ajustar para controlar el nivel de suavizado.

- bigram_counts y self.unigram_counts: Diccionarios para almacenar los conteos de bigramas y unigrams, respectivamente.

- vocabulario: Conjunto de palabras únicas que componen el vocabulario del modelo.

Tiene el metodo entrenar que recibe el corpus y hace el conteo de bigramas y unigrams en el corpus proporcionado recorriendo cada par de palabras consecutivas en el corpus y contando cada par como un bigrama y cada palabra individual como un unigrama

Tiene el metodo calcular Probabilidades suavizadas que imprime la probabilidad de cada bigrama con el suavizado aplicado.

tiene el metodo calcular_probabilidad_bigrama el cual calcula calcula la probabilidad suavizada de un bigrama específico (contexto, palabra) usando Add-k.



In [ ]:
#Modelo con suavizado Add-k
class ModeloNGrama_Addk:
    def __init__(self, n: int, k: float):
        self.n = n  # cuando es bigrama, n=2
        self.k = k  # parámetro de suavizado add-k
        self.bigram_counts = {}
        self.unigram_counts = {}
        self.vocabulario = {'<s>', '</s>', 'a', 'all', 'model', 'models', 'some', 'useful', 'wrong'}
        self.total_bigrams = 0

    def entrenar(self, corpus: List[str]):
        # Conteo de bigramas y unigrams
        for i in range(len(corpus) - 1):
            bigram = (corpus[i], corpus[i + 1])
            unigram = corpus[i]

            # Conteo de bigramas
            if bigram in self.bigram_counts:
                self.bigram_counts[bigram] += 1
            else:
                self.bigram_counts[bigram] = 1
            # Conteo de unigrams
            if unigram in self.unigram_counts:
                self.unigram_counts[unigram] += 1
            else:
                self.unigram_counts[unigram] = 1

        # Contar el último unigram
        last_word = corpus[-1]
        if last_word in self.unigram_counts:
            self.unigram_counts[last_word] += 1
        else:
            self.unigram_counts[last_word] = 1

    def calcular_probabilidades_suavizadas(self):
        # Tamaño del vocabulario
        V = len(self.unigram_counts)
        add_k_probabilities = {}

        print(f"Probabilidades de bigramas (con suavizado add-k) con k = {k}")
        for bigram, bigram_count in self.bigram_counts.items():
            w_n_1 = bigram[0]
            # Aplicando la ecuación P_Add-k(w_n | w_n-1) = (C(w_n-1, w_n) + k) / (C(w_n-1) + k * V)
            probabilidad = (bigram_count + self.k) / (self.unigram_counts[w_n_1] + self.k * V)
            add_k_probabilities[bigram] = probabilidad
            print(f"P({bigram[1]} | {bigram[0]}) = {probabilidad:.4f}")

        return add_k_probabilities

    def calcular_probabilidad_bigrama(self, contexto: str, palabra: str):
        V = len(self.unigram_counts)
        bigram = (contexto, palabra)

        # Aplicando la ecuación para bigramas no vistos con suavizado add-k
        cuenta_bigrama = self.bigram_counts.get(bigram, 0) + self.k
        cuenta_contexto = self.unigram_counts.get(contexto, 0) + self.k * V
        probabilidad = cuenta_bigrama / cuenta_contexto
        print("\nLa probabilidad de el bigrama ingresado es")
        print(f"P({palabra} | {contexto}) = {probabilidad:.4f}")
        return probabilidad


In [ ]:
k = 0.05  #Primera ejecucion con k = 0.05
modelo_bigrama = ModeloNGrama_Addk(n = 2, k = k)
modelo_bigrama.entrenar(corpus)
modelo_bigrama.calcular_probabilidades_suavizadas()

# Calcular probabilidad de un bigrama específico
modelo_bigrama.calcular_probabilidad_bigrama('a', 'models')

Probabilidades de bigramas (con suavizado add-k) con k = 0.05
P(models | all) = 0.7241
P(are | models) = 0.8367
P(wrong | are) = 0.8367
P(a | wrong) = 0.3043
P(model | a) = 0.7241
P(is | model) = 0.7241
P(wrong | is) = 0.7241
P(some | wrong) = 0.3043
P(useful | some) = 0.7241
P(models | useful) = 0.7241

La probabilidad de el bigrama ingresado es
P(models | a) = 0.0345


0.034482758620689655

In [ ]:
k = 0.15  #Segunda ejecucion con k = 0.15
modelo_bigrama = ModeloNGrama_Addk(n = 2, k = k)
modelo_bigrama.entrenar(corpus)
modelo_bigrama.calcular_probabilidades_suavizadas()

# Calcular probabilidad de un bigrama específico
modelo_bigrama.calcular_probabilidad_bigrama('a', 'models')

Probabilidades de bigramas (con suavizado add-k) con k = 0.15
P(models | all) = 0.4894
P(are | models) = 0.6418
P(wrong | are) = 0.6418
P(a | wrong) = 0.2644
P(model | a) = 0.4894
P(is | model) = 0.4894
P(wrong | is) = 0.4894
P(some | wrong) = 0.2644
P(useful | some) = 0.4894
P(models | useful) = 0.4894

La probabilidad de el bigrama ingresado es
P(models | a) = 0.0638


0.06382978723404256

En este caso, k =0.05, lo que representa un valor pequeño de suavizado.
Las probabilidades calculadas reflejan una distribución donde las palabras que aparecen con frecuencia juntas en el corpus tienen una mayor probabilidad

Con k=0.15, palabras que aparecen con frecuencia juntas en el corpus tienen una mayor probabilidad.

En general Un valor más alto de k genera una distribución de probabilidad más uniforme, lo que significa que las probabilidades de los bigramas más frecuentes disminuyen, mientras que las de los bigramas menos comunes o no vistos aumenta, por ejemplo con el bigrama p(models|a) aumenta a 0.0638, indicando que el suavizado ha asignado una probabilidad mayor a los bigramas no observados directamente en el corpus.

#Modelo backoff

Este codigo tienen como objetivo representar un modelo backoff, de tal manera que cuando un bigrama (par de palabras consecutivas) no se ha visto en el corpus, en lugar de asignarle una probabilidad de cero, el modelo "retrocede" y utiliza la probabilidad de unigram (probabilidad de la palabra independiente) para el cálculo, ajustada con un factor de normalización.

Atributos:
- bigram_counts: Almacena el conteo de cada bigrama en el corpus.
- unigram_counts: Almacena el conteo de cada unigram en el corpus.
- total_tokens: Cuenta el total de palabras (tokens) en el corpus.

Tiene el metodo 'entrenar' que cuenta las apariciones de los unigramas y de los bigramas recorriendo cada palabra en el corpus y si hay una palabra sieguiete, el par de palabras se cuentaa como un bigrama en bigram.counts.

Tiene el metodo prob_bigrama que calcula todal las probabilidades de os bigramas existentes en el corpus mediante la formula Conteo bigama/ conteo unigrama (V).

Tiene el metodo probabilidad unigrama, que calula la probabilidad de cadad unigrama con la formula (Conteo unigrama/total tokens) y si no hay asigna una probabilidad muy baja 1/ total tokens

Tiene el metodod alpha, que calcula el valor de normalizacion para el backoff, este factor alpha se asegura que la probabilidad total siga sumando 1 cuando se retrocede en el modelo unigrama, ademas si w1 tiene conteos en el corpus, calcula α(w1) restando la suma de probabilidades de bigramas y dividiendo por la suma de probabilidades de unigrams.

Tiene el metodo calcular_probabilidad_bigrama, que calcula la probabilidad suavizada de todos los bigramas en el corpus, usando el principio de si el bigrama tiene un conteo se usa su probabilidad, caso contrario se hace el retroceso haciendo que la probabilidad(w2) sea gual al del unigrama multiplado por el alpha(w1), imprime la probabilidad y retorna la probabilidad

Tiene el metodo calcular_probabilidades_suavizadas que devuele la probabilidad de todos los bigramas y retorna todas las probabilidades

In [ ]:
#modelo backoff
class ModeloBackoffBigram:
    def __init__(self):
        self.bigram_counts = collections.Counter()
        self.unigram_counts = collections.Counter()
        self.total_tokens = 0
    def entrenar(self, corpus: List[str]):
        # Conteo de unigrams y bigrams
        for i in range(len(corpus)):
            unigram = corpus[i]
            self.unigram_counts[unigram] += 1
            self.total_tokens += 1

            if i < len(corpus) - 1:
                bigram = (corpus[i], corpus[i + 1])
                self.bigram_counts[bigram] += 1

    def prob_bigrama(self, w1: str, w2: str):
        # Si el bigrama (w1, w2) tiene un conteo, calculamos su probabilidad directamente
        bigram = (w1, w2)
        if self.bigram_counts[bigram] > 0:
            return self.bigram_counts[bigram] / self.unigram_counts[w1]
        else:
            return None

    def prob_unigram(self, w: str):
        # Probabilidad del unigram
        return self.unigram_counts[w] / self.total_tokens if w in self.unigram_counts else 1 / self.total_tokens

    def alpha(self, w1: str):
        # Factor α(w1) para el retroceso
        if self.unigram_counts[w1] > 0:
            return (1 - sum(self.prob_bigrama(w1, w2) or 0 for w2 in self.unigram_counts)) / \
                   (1 - sum(self.prob_unigram(w2) for w2 in self.unigram_counts))
        else:
            return 1

    def calcular_probabilidad_bigrama(self, w1: str, w2: str):
        # Cálculo de la probabilidad
        bigram_prob = self.prob_bigrama(w1, w2)
        probabilidad = 0
        if bigram_prob is not None:
            probabilidad = bigram_prob
        else:
            # Si no existe el bigrama, aplicamos el retroceso
            probabilidad = self.alpha(w1) * self.prob_unigram(w2)

        print(f"P({w2} | {w1}) = {probabilidad:.4f}")
        return probabilidad

    def calcular_probabilidades_suavizadas(self) -> Dict[Tuple[str, str], float]:
        # Imprime y devuelve las probabilidades de todos los bigramas en el corpus con suavizado backoff
        print("Probabilidades con el suavizado backoff")
        probabilidades_bigrama = {}
        for (w1, w2) in self.bigram_counts:
            probabilidad = self.calcular_probabilidad_bigrama(w1, w2)
            probabilidades_bigrama[(w1, w2)] = probabilidad

        return probabilidades_bigrama

In [ ]:
modelo_bigrama = ModeloBackoffBigram()
modelo_bigrama.entrenar(corpus)
modelo_bigrama.calcular_probabilidades_suavizadas()

# Calcular probabilidad de un bigrama específico
print("\nusando el bigrama no conocido")
modelo_bigrama.calcular_probabilidad_bigrama('a', 'models')

Probabilidades con el suavizado backoff
P(models | all) = 1.0000
P(are | models) = 1.0000
P(wrong | are) = 1.0000
P(a | wrong) = 0.3333
P(model | a) = 1.0000
P(is | model) = 1.0000
P(wrong | is) = 1.0000
P(some | wrong) = 0.3333
P(useful | some) = 1.0000
P(models | useful) = 1.0000

usando el bigrama no conocido
P(models | a) = 0.0000


0.0

se muestran los resultados del modelo de Bigramas con Suavizado Backoff para varios bigramas en el corpus. La lógica del modelo es asignar probabilidades basadas en los conteos observados de bigramas, y, si un bigrama no se encuentra en el corpus, el modelo debería retroceder al modelo unigram, ajustando la probabilidad con el factor de normalización α(w1).

Para P(a|models) la probabilidad sale 0 porque α(w1) tambien e 0

#Stupid Backoff

de "Stupid Backoff" (retroceso simple), si un bigrama no se encuentra en el corpus, el modelo utiliza la probabilidad de unigram de la segunda palabra, pero multiplicada por un factor de descuento (normalmente, un valor menor que 1). Este método es más simple que otros métodos de suavizado y retroceso, ya que no intenta normalizar las probabilidades para que sumen exactamente 1.

Tiene los siguientes atributos:
- bigram_counts: Almacena el conteo de cada bigrama en el corpus.
- unigram_counts: Almacena el conteo de cada unigram en el corpus.
- total_tokens: Cuenta el total de palabras (tokens) en el corpus.
- discount: Factor de descuento para el suavizado backoff. Aquí se establece en 0.4 (un valor típico), lo cual significa que si un bigrama no se observa, se recurre al unigram de la segunda palabra con un descuento del 60%.

Tiene el metodo "Entrenar" que lo que hace es contar bigramas y unigramas enel corpus, recorriendo cada palabra individal y contandolo como un unigrama y cada par consecuitvo como un bigrama y los almacena en sus cuentas respectivas.

Tiene el metodo probabilidad bigrama que devuelve la probabilidad de un bigrama si es que tiene cuenta sino devuelve un nulo

Tiene el metodo prob_unigrama, que calcula la probabilidad de una sola palabra basado en el conteo del corpus

Tiene el metodo Calcular_Probabilidad_bigrama que devuelve la probabilidad con el suavizado stupid backoff, es decir si hay la probabilidad bigrama devuelve la probabilidad del bigrama, caso contrario devuelve la probabilidad unigrama por la tasa de descuento e imprime la probabilidad.

Finalmente esta "calcular_probabilidades_suavizadas" calcula las probabilidades de todos los bigramas del corpus y se ayuda del metodo calcular_probabilidad_bigrama

In [ ]:
#StupidBackoff
class ModeloStupidBackoffBigram:
    def __init__(self, discount: float = 0.4):
        self.bigram_counts = collections.Counter()
        self.unigram_counts = collections.Counter()
        self.total_tokens = 0
        self.discount = discount  # Factor de descuento

    def entrenar(self, corpus: List[str]):
        # Conteo de unigrams y bigrams
        for i in range(len(corpus)):
            unigram = corpus[i]
            self.unigram_counts[unigram] += 1
            self.total_tokens += 1

            if i < len(corpus) - 1:
                bigram = (corpus[i], corpus[i + 1])
                self.bigram_counts[bigram] += 1

    def prob_bigrama(self, w1: str, w2: str):
        bigram = (w1, w2)
        if self.bigram_counts[bigram] > 0:
            return self.bigram_counts[bigram] / self.unigram_counts[w1]
        else:
            return None

    def prob_unigram(self, w: str):
        # Probabilidad del unigram
        return self.unigram_counts[w] / self.total_tokens if w in self.unigram_counts else 1 / self.total_tokens

    def calcular_probabilidad_bigrama(self, w1: str, w2: str):
        # Cálculo de la probabilidad con stupid backoff
        bigram_prob = self.prob_bigrama(w1, w2)
        probabilidad = 0
        if bigram_prob is not None:
            probabilidad = bigram_prob
        else:
            # Si no existe el bigrama, aplicamos el retroceso
            probabilidad = self.discount * self.prob_unigram(w2)

        print(f"P({w2} | {w1}) = {probabilidad:.4f}")
        return probabilidad

    def calcular_probabilidades_suavizadas(self) -> Dict[Tuple[str, str], float]:
        # Imprime y devuelve las probabilidades de todos los bigramas en el corpus
        print("Probabilidades con el suavizado stupid backoff")
        probabilidades_bigrama = {}
        for (w1, w2) in self.bigram_counts:
            probabilidad = self.calcular_probabilidad_bigrama(w1, w2)
            probabilidades_bigrama[(w1, w2)] = probabilidad

        return probabilidades_bigrama

In [ ]:
modelo_bigrama = ModeloStupidBackoffBigram()
modelo_bigrama.entrenar(corpus)
probabilidades = modelo_bigrama.calcular_probabilidades_suavizadas()

# Calcular probabilidad de un bigrama específico
print("\nusando el bigrama no conocido")
probabilidad_models_a = modelo_bigrama.calcular_probabilidad_bigrama('a', 'models')

Probabilidades con el suavizado stupid backoff
P(models | all) = 1.0000
P(are | models) = 1.0000
P(wrong | are) = 1.0000
P(a | wrong) = 0.3333
P(model | a) = 1.0000
P(is | model) = 1.0000
P(wrong | is) = 1.0000
P(some | wrong) = 0.3333
P(useful | some) = 1.0000
P(models | useful) = 1.0000

usando el bigrama no conocido
P(models | a) = 0.0615


Aqui se muestran los resultados del cálculo de probabilidades de bigramas usando el suavizado Stupid Backoff con un descuento aplicado.

Estas probabilidades de 1.0 indican que, en el corpus, cuando la primera palabra del bigrama (por ejemplo, "all") aparece, siempre es seguida por la segunda palabra (por ejemplo, "models"). Esto sugiere que el corpus tiene patrones muy repetitivos o es relativamente pequeño, lo que permite que algunos bigramas tengan una probabilidad de ocurrencia del 100%.

Para los bigramas no observados en el corpus, el modelo usa el Stupid Backoff multiplicando la probabilidad del unigram de la segunda palabra por el factor de descuento como P(a∣wrong)=0.3333, esto significa que, al no encontrar estos bigramas en el corpus, el modelo recurre a la probabilidad de unigram y la ajusta con el descuento para no asignar una probabilidad de cero.

Para el bigrama no conocidoel modelo no asigna cero sino que lo penaliza con el descuento, dándole una probabilidad más baja en comparación con los bigramas observados.

#Good Turning

Este código implementa un modelo de Good-Turing para ajustar las probabilidades de unigramas en un corpus de texto. Good-Turing es un método de suavizado que ayuda a ajustar las probabilidades para eventos raros o no observados en un corpus al redistribuir la probabilidad de eventos observados con conteos bajos hacia aquellos que no se observaron. Este enfoque es útil en procesamiento de lenguaje natural, especialmente en casos donde no todos los posibles unigramas están presentes en el corpus.

Tiene los atributos:
- corpus: El corpus de texto como una lista de palabras.
- unigram_counts: Conteo de cada unigram (palabra) en el corpus.
- total_tokens: Número total de tokens en el corpus.
- count_of_counts: Un diccionario que cuenta cuántas palabras tienen cada conteo específico.
- adjusted_counts: Diccionario para almacenar los conteos ajustados según el método de Good-Turing.
- unigram_probabilities: Diccionario para almacenar las probabilidades de unigramas ajustadas
- unigram_probabilities_norm: Diccionario para almacenar las probabilidades ajustadas después de la normalización.

Tiene el metodo "calcular_r_nr" que imprime los conteos de unigramas y el "conteo de conteos" Nr(la cantidad de unigramas que tienen cada frecuencia r) y devuelve dos diccionarios uno con los conteos de unigram y otro con los conteos de conteos.

Tiene el metodo "ajustar_conteos_good_turing" el cual ajusta conteos de cada unigrama utilizando la formula Good-turing, si existe un valor N(r+1) el conteo se ajusta usando la formula, si no hay N(r+1) se mantiene el conteo original e imprime los conteos ajustado para unigramas con conteos menores a 3.

Tiene el metodo "calcular_probabilidades_unigramas" Este método calcula las probabilidades ajustadas de unigramas según el método de Good-Turing y, para los unigrams más frecuentes (con el valor máximo de r), utiliza el MLE (Estimación de Máxima Verosimilitud).

Tiene el metodo "normalizar_probabilidades" el cual normaliza las probabilidades ajustadas para asegurar que la suma sea 1, para ello calcula la suma de las probabilidades ajustadas. Si la suma no es 1, aplica un factor de normalización e imprime las probabilidad.


In [ ]:
#Good Turing
class ModeloGoodTuring:
    def __init__(self, corpus: List[str]):
        self.corpus = corpus
        self.unigram_counts = collections.Counter(corpus)
        self.total_tokens = sum(self.unigram_counts.values())
        self.count_of_counts = collections.Counter(self.unigram_counts.values())
        self.adjusted_counts = {}
        self.unigram_probabilities = {}
        self.unigram_probabilities_norm = {}

    def calcular_r_y_nr(self) -> Tuple[Dict[str, int], Dict[int, int]]:
        # Impresión de los conteos de unigrams (r) y de conteo de conteos (Nr)
        print("Conteos de unigrams (r):")
        for unigram, count in self.unigram_counts.items():
            print(f"  r({unigram}) = {count}")

        print("\nConteo de conteos (Nr):")
        for r, Nr in self.count_of_counts.items():
            print(f"  N_{r} = {Nr}")

        return dict(self.unigram_counts), dict(self.count_of_counts)

    def ajustar_conteos_good_turing(self):
        # Ajustar los conteos usando la fórmula de Good-Turing
        for unigram, count in self.unigram_counts.items():
            if count + 1 in self.count_of_counts:
                self.adjusted_counts[unigram] = (count + 1) * (self.count_of_counts[count + 1] / self.count_of_counts[count])
            else:
                self.adjusted_counts[unigram] = count  # Mantener el conteo original si no hay ajuste

        # Impresión de los conteos ajustados (Good-Turing)
        print("\nConteos ajustados (Good-Turing):")
        for unigram, adjusted_count in self.adjusted_counts.items():
            if self.unigram_counts[unigram] < 3:
              print(f"  C*({unigram}) = {adjusted_count:.4f}")

    def calcular_probabilidades_unigramas(self):
        # Encuentra el valor máximo de frecuencia r en el corpus
        r_max = max(self.unigram_counts.values())

        # Calcular las probabilidades ajustadas de Good-Turing para todos los unigrams r<3
        print("\nProbabilidades ajustadas de unigrams (Good-Turing):")
        for unigram, count in self.adjusted_counts.items():
            if self.unigram_counts[unigram] != r_max:
                probability = count / self.total_tokens
                self.unigram_probabilities[unigram] = probability
                print(f"  P({unigram}) = {probability:.4f}")

        # Calcular la probabilidad de unigrams con la frecuencia máxima usando MLE
        print(f"\nProbabilidades MLE para el valor máximo de r = {r_max}:")
        for unigram, count in self.unigram_counts.items():
            if count == r_max:
                probability = count / self.total_tokens
                self.unigram_probabilities[unigram] = probability
                print(f"  P({unigram}) = {probability:.4f} (MLE)")

        return self.unigram_probabilities

    '''
    La normalizacion que se va a utiliar es La normalización por suma unitaria
    ya qes la forma más sencilla de ajustar la suma sin cambiar demasiado
    las probabilidades individuales sin necesidad de recalcular los conteos ajustados.
    '''

    def normalizar_probabilidades(self):
        # Sumar las probabilidades antes de la normalización
        suma_probabilidades = sum(self.unigram_probabilities.values())
        print(f"\nSuma de las probabilidades antes de normalizar: {suma_probabilidades:.4f}")

        # Normalizar si la suma no es 1 y guardar en unigram_probabilities_norm
        if suma_probabilidades != 1:
            factor_normalizacion = 1 / suma_probabilidades
            self.unigram_probabilities_norm = {unigram: prob * factor_normalizacion for unigram, prob in self.unigram_probabilities.items()}
            print("\nProbabilidades después de normalizar:")
            for unigram, prob in self.unigram_probabilities_norm.items():
                print(f"  P({unigram}) = {prob:.4f}")

        # Sumar las probabilidades despues de la normalización
        print(f"\nSuma de las probabilidades despues de normalizar: {sum(self.unigram_probabilities_norm.values()):.4f}")

        return self.unigram_probabilities_norm
    '''
    Tambien se intento hacer la normalizacion de la ecuacion 6.14 del libro Foundations of Statistical Natural Language Processing
    pero para el ejemplo le da mucha probabilidad a los valores no observados a mi parecer

    def normalizar_probabilidades(self):
        # Sumar las probabilidades antes de la normalización
        suma_probabilidades = sum(self.unigram_probabilities.values())
        print(f"\nSuma de las probabilidades antes de normalizar: {suma_probabilidades:.4f}")

        # Normalizar si la suma no es 1 y guardar en unigram_probabilities_norm
        if suma_probabilidades != 1:
            # Probabilidad para los eventos no observados (r = 0)
            unseen_prob = self.count_of_counts.get(1, 0) / self.total_tokens
            print(f"\nProbabilidad para eventos no observados (Good-Turing): P(unseen) = {unseen_prob:.4f}")

            # Ajustar las probabilidades para que sumen 1, redistribuyendo
            prob_observed_sum = sum(self.unigram_probabilities.values())
            ajuste_factor = (1 - unseen_prob) / prob_observed_sum

            # Aplicar el ajuste a las probabilidades de los eventos observados
            self.unigram_probabilities_norm = {unigram: prob * ajuste_factor for unigram, prob in self.unigram_probabilities.items()}
            self.unigram_probabilities_norm["<no_observado>"] = unseen_prob  # Incluir la probabilidad para eventos no observados

            print("\nProbabilidades después de la normalización:")
            for unigram, prob in self.unigram_probabilities_norm.items():
                print(f"  P({unigram}) = {prob:.4f}")

        # Sumar las probabilidades despues de la normalización
        print(f"\nSuma de las probabilidades despues de normalizar: {sum(self.unigram_probabilities_norm.values()):.4f}")

        return self.unigram_probabilities_norm

        Probabilidades después de la normalización:
          P(all) = 0.0359
          P(models) = 0.0808
          P(are) = 0.0808
          P(a) = 0.0359
          P(model) = 0.0359
          P(is) = 0.0359
          P(some) = 0.0359
          P(useful) = 0.0359
          P(wrong) = 0.1615
          P(<no_observado>) = 0.4615

        Suma de las probabilidades despues de normalizar: 1.0000
    '''

In [ ]:
modelo = ModeloGoodTuring(corpus)
modelo.calcular_r_y_nr()
#Probbilidades para r<3
print("\nCr y Probabilidades para r<3")
modelo.ajustar_conteos_good_turing()
modelo.calcular_probabilidades_unigramas()
modelo.normalizar_probabilidades()
print()

Conteos de unigrams (r):
  r(all) = 1
  r(models) = 2
  r(are) = 2
  r(wrong) = 3
  r(a) = 1
  r(model) = 1
  r(is) = 1
  r(some) = 1
  r(useful) = 1

Conteo de conteos (Nr):
  N_1 = 6
  N_2 = 2
  N_3 = 1

Cr y Probabilidades para r<3

Conteos ajustados (Good-Turing):
  C*(all) = 0.6667
  C*(models) = 1.5000
  C*(are) = 1.5000
  C*(a) = 0.6667
  C*(model) = 0.6667
  C*(is) = 0.6667
  C*(some) = 0.6667
  C*(useful) = 0.6667

Probabilidades ajustadas de unigrams (Good-Turing):
  P(all) = 0.0513
  P(models) = 0.1154
  P(are) = 0.1154
  P(a) = 0.0513
  P(model) = 0.0513
  P(is) = 0.0513
  P(some) = 0.0513
  P(useful) = 0.0513

Probabilidades MLE para el valor máximo de r = 3:
  P(wrong) = 0.2308 (MLE)

Suma de las probabilidades antes de normalizar: 0.7692

Probabilidades después de normalizar:
  P(all) = 0.0667
  P(models) = 0.1500
  P(are) = 0.1500
  P(a) = 0.0667
  P(model) = 0.0667
  P(is) = 0.0667
  P(some) = 0.0667
  P(useful) = 0.0667
  P(wrong) = 0.3000

Suma de las probabilidades 

En Conteos de Unigrams (r): Cada palabra tiene un conteo r que representa cuántas veces aparece en el corpus.

En Conteo de Conteos (Nr): representa cuántas palabras tienen un cierto conteo r.

Conteos Ajustados (Good-Turing):Los conteos ajustados redistribuyen la probabilidad de palabras con conteos bajos hacia palabras no observadas.Los conteos ajustados se calculan con la fórmula de Good-Turing C* = (r+1)*((N_r+1))/Nr.

Probabilidades Ajustadas de Unigrams (Good-Turing): Calculan las probabilidades ajustadas dividiendo el conteo ajustado entre el total de tokens.

Probabilidades MLE para r=3: La palabra "wrong", que tiene el valor máximo r=3, utiliza la Estimación de Máxima Verosimilitud (MLE).

Suma de Probabilidades antes de Normalizar: La suma de las probabilidades antes de la normalización es 0.7692, lo cual indica que las probabilidades ajustadas no suman exactamente 1.

Probabilidades después de Normalizar: Se aplica un factor de normalización para ajustar las probabilidades de manera que su suma sea 1.

Para cumplir lo que se pide en el item i se debe hacer una modificacion para casos donde Nr = 0.

Se puede asignar un valor para N5 que utilice entre N4 y los valores que siguen suavizando el cálculo de probabilidades en corpus grandes. Esta implementacion se hace actualizando el método ajustar_conteos_good_turing para considerar los casos en que Nr = 0 y asigne el valor estimado.

En el primer código, se verifica si el conteo N5 está ausente en count_of_counts.
Si N5 falta, se le asigna un valor aproximado calculado como 0.9 * N4, que es un valor suavizado para evitar que N5 sea cero.
Este enfoque proporciona un ajuste en la fórmula de Good-Turing cuando hay valores de Nr ausentes, lo cual es útil en corpus grandes donde ciertos conteos pueden no aparecer.

In [9]:
import collections
from typing import List, Dict, Tuple
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Vocabulario
vocabulario = {'<s>', '</s>', 'a', 'all', 'model', 'models', 'some', 'useful', 'wrong'}

#corpus
frase_1 =  "all models are wrong"
frase_2 =  "a model is wrong wrong wrong"
frase_3 =  "some useful models are wrong"
lista_frases = [[frase_1], [frase_2], [frase_3]]

def construir_corpus(lista_frases: List[str]) -> List[str]:
    """
    Constructs a corpus as a flat list of tokens from a list of sentences.
    """
    corpus = []
    for frase in lista_frases:
        sentence = frase[0]
        tokens = word_tokenize(sentence)
        corpus.extend(tokens) # Extend corpus with tokens of this sentence
    return corpus

corpus = construir_corpus(lista_frases)
print(corpus)

['all', 'models', 'are', 'wrong', 'a', 'model', 'is', 'wrong', 'wrong', 'wrong', 'some', 'useful', 'models', 'are', 'wrong']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
class ModeloGoodTuring:
    def __init__(self, corpus: List[str]):
        self.corpus = corpus
        self.unigram_counts = collections.Counter(corpus)
        self.total_tokens = sum(self.unigram_counts.values())
        self.count_of_counts = collections.Counter(self.unigram_counts.values())
        self.adjusted_counts = {}
        self.unigram_probabilities = {}
        self.unigram_probabilities_norm = {}

    def calcular_r_y_nr(self) -> Tuple[Dict[str, int], Dict[int, int]]:
        # Impresión de los conteos de unigrams (r) y de conteo de conteos (Nr)
        print("Conteos de unigrams (r):")
        for unigram, count in self.unigram_counts.items():
            print(f"  r({unigram}) = {count}")

        print("\nConteo de conteos (Nr):")
        for r, Nr in self.count_of_counts.items():
            print(f"  N_{r} = {Nr}")

        return dict(self.unigram_counts), dict(self.count_of_counts)

    def ajustar_conteos_good_turing(self):
        # Ajustar los conteos usando la fórmula de Good-Turing
        max_r = max(self.unigram_counts.values())

        # Estimar N5 si falta
        if 5 not in self.count_of_counts:
            # Suavizado: Usa un valor promedio entre N4 y otros valores si es posible
            self.count_of_counts[5] = self.count_of_counts.get(4, 1) * 0.9  # Valor aproximado para N5

        for unigram, count in self.unigram_counts.items():
            if count + 1 in self.count_of_counts:
                self.adjusted_counts[unigram] = (count + 1) * (self.count_of_counts[count + 1] / self.count_of_counts[count])
            else:
                self.adjusted_counts[unigram] = count  # Mantener el conteo original si no hay ajuste

        # Impresión de los conteos ajustados (Good-Turing)
        print("\nConteos ajustados (Good-Turing):")
        for unigram, adjusted_count in self.adjusted_counts.items():
            if self.unigram_counts[unigram] < 3:
                print(f"  C*({unigram}) = {adjusted_count:.4f}")

    def calcular_probabilidades_unigramas(self):
        # Encuentra el valor máximo de frecuencia r en el corpus
        r_max = max(self.unigram_counts.values())

        # Calcular las probabilidades ajustadas de Good-Turing para todos los unigrams r<3
        print("\nProbabilidades ajustadas de unigrams (Good-Turing):")
        for unigram, count in self.adjusted_counts.items():
            if self.unigram_counts[unigram] != r_max:
                probability = count / self.total_tokens
                self.unigram_probabilities[unigram] = probability
                print(f"  P({unigram}) = {probability:.4f}")

        # Calcular la probabilidad de unigrams con la frecuencia máxima usando MLE
        print(f"\nProbabilidades MLE para el valor máximo de r = {r_max}:")
        for unigram, count in self.unigram_counts.items():
            if count == r_max:
                probability = count / self.total_tokens
                self.unigram_probabilities[unigram] = probability
                print(f"  P({unigram}) = {probability:.4f} (MLE)")

        return self.unigram_probabilities

    '''
    La normalizacion que se va a utiliar es La normalización por suma unitaria
    ya qes la forma más sencilla de ajustar la suma sin cambiar demasiado
    las probabilidades individuales sin necesidad de recalcular los conteos ajustados.
    '''

    def normalizar_probabilidades(self):
        # Sumar las probabilidades antes de la normalización
        suma_probabilidades = sum(self.unigram_probabilities.values())
        print(f"\nSuma de las probabilidades antes de normalizar: {suma_probabilidades:.4f}")

        # Normalizar si la suma no es 1 y guardar en unigram_probabilities_norm
        if suma_probabilidades != 1:
            factor_normalizacion = 1 / suma_probabilidades
            self.unigram_probabilities_norm = {unigram: prob * factor_normalizacion for unigram, prob in self.unigram_probabilities.items()}
            print("\nProbabilidades después de normalizar:")
            for unigram, prob in self.unigram_probabilities_norm.items():
                print(f"  P({unigram}) = {prob:.4f}")

        # Sumar las probabilidades despues de la normalización
        print(f"\nSuma de las probabilidades despues de normalizar: {sum(self.unigram_probabilities_norm.values()):.4f}")

        return self.unigram_probabilities_norm

In [11]:
modelo = ModeloGoodTuring(corpus)
modelo.calcular_r_y_nr()
#Probbilidades para r<3
print("\nCr y Probabilidades para r<3")
modelo.ajustar_conteos_good_turing()
modelo.calcular_probabilidades_unigramas()
modelo.normalizar_probabilidades()
print()

Conteos de unigrams (r):
  r(all) = 1
  r(models) = 2
  r(are) = 2
  r(wrong) = 5
  r(a) = 1
  r(model) = 1
  r(is) = 1
  r(some) = 1
  r(useful) = 1

Conteo de conteos (Nr):
  N_1 = 6
  N_2 = 2
  N_5 = 1

Cr y Probabilidades para r<3

Conteos ajustados (Good-Turing):
  C*(all) = 0.6667
  C*(models) = 2.0000
  C*(are) = 2.0000
  C*(a) = 0.6667
  C*(model) = 0.6667
  C*(is) = 0.6667
  C*(some) = 0.6667
  C*(useful) = 0.6667

Probabilidades ajustadas de unigrams (Good-Turing):
  P(all) = 0.0444
  P(models) = 0.1333
  P(are) = 0.1333
  P(a) = 0.0444
  P(model) = 0.0444
  P(is) = 0.0444
  P(some) = 0.0444
  P(useful) = 0.0444

Probabilidades MLE para el valor máximo de r = 5:
  P(wrong) = 0.3333 (MLE)

Suma de las probabilidades antes de normalizar: 0.8667

Probabilidades después de normalizar:
  P(all) = 0.0513
  P(models) = 0.1538
  P(are) = 0.1538
  P(a) = 0.0513
  P(model) = 0.0513
  P(is) = 0.0513
  P(some) = 0.0513
  P(useful) = 0.0513
  P(wrong) = 0.3846

Suma de las probabilidades 

Ejercicio 2

Este codigo recorre la parte de preparacion para la primera parte que es el preprocesamiento de texto del ejercicio, para ello es importante tener el corpus infopankki.en-es.es en el entorno de ejecucion.

Esta el metodo para leer archivo ubicado en la ruta especificada y devuelve una lista de líneas de texto. La codificación utf-8 se asegura de manejar caracteres especiales.

Luego esta la funcion tokenizar_linea que Convierte el texto a minúsculas, separa palabras y omite caracteres no alfabéticos, filtra las stopwords, devolviendo solo palabras relevantes en tokens.

Luego esta la funcion stemizar_tokens que Reduce palabras a su raíz (stem) eliminando sufijos comunes en español, para ello verifica si la palabra termina con alguno de los sufijos en sufijos_comunes y, si es así, lo elimina y devuelve la palabra sin sufijo (o la palabra original si no tiene sufijo).

Tiene la funcion "filtrar_palabras_raras" que retiene solo aquellas que aparecen al menos frecuencia_minima veces, para ello se pasa un contador, recorre el corpus y elimina las palabras cuya frecuencia sea mayor o igual a la fracuencia minima.

Finalmente es la funcion preprocesar_archivo que aplica todo lo dicho anteriormente, tokenizacion, steming y filtradoo y regresa una lista con los tokens del corpus.

In [ ]:
import re
import random
from collections import Counter


def leer_archivo(ruta):
    with open(ruta, 'r', encoding='utf-8') as file:
        return file.readlines()

ruta = 'infopankki.en-es.es'
archivo = leer_archivo(ruta)

#tokenizador
def tokenizar_linea(texto):
  stopwords = {"el", "la", "los", "las", "un", "una", "unos", "unas", "de", "a", "y", "en", "que", "por", "con"}
  # Convertir a minúsculas.
  texto = texto.lower()
  palabras = []
  palabra_actual = ""
  for char in texto:
    if char.isalpha():  # Si es letra o número, añadir al token actual
          palabra_actual += char
    else:
      if palabra_actual:  # Si hay una palabra construida, añadirla y limpiar el token
        palabras.append(palabra_actual)
        palabra_actual = ""
  if palabra_actual:  # Añadir la última palabra si hay una pendiente
    palabras.append(palabra_actual)

  # Filtrar stopwords
  tokens = [palabra for palabra in palabras if palabra not in stopwords]

  return tokens

#stemizador
def stemizar_tokens(palabra):
  sufijos_comunes = ["ando", "iendo", "ar", "er", "ir", "ado", "ido", "es", "s", "mente", "ción", "ciones", "dor", "dora"]
  for sufijo in sufijos_comunes:
    if palabra.endswith(sufijo):
      # Quitar el sufijo y devolver la palabra modificada
      return palabra[:-len(sufijo)]

  return palabra

# Filtrado de palabras raras
def filtrar_palabras_raras(tokens, frecuencia_minima=5):
  # Contar frecuencias de todas las palabras
  contador = Counter(tokens)
  # Filtrar palabras que cumplan con la frecuencia
  palabras_filtradas = [palabra for palabra in tokens if contador[palabra] >= frecuencia_minima]
  return palabras_filtradas


#Preprosesamiento de texto
def preprocesar_archivo(archivo, num_lineas=1000):
    preprocesado = []
    lineas = [] #limitare el numero de lineas porque si uso todo el corpus se demora demasiado
    for muestra in random.sample(archivo, num_lineas):
      lineas.append(muestra)

    for linea in lineas:
        tokens = tokenizar_linea(linea)  # Tokenizar y eliminar stopwords
        stems = [stemizar_tokens(token) for token in tokens]  # Stemizar
        stems_filtrados = filtrar_palabras_raras(stems)  # Filtrar palabras raras
        preprocesado.append(stems)
    return preprocesado

corpus_2 = preprocesar_archivo(archivo)
corpus_2

[['ejemplo',
  'matrimonio',
  'haya',
  'dur',
  'meno',
  'cinco',
  'año',
  'bien',
  'no',
  'se',
  'reparten',
  'necesaria',
  'part',
  'igual'],
 ['deberá',
  'abon',
  'pago',
  'momento',
  'solicit',
  'prórroga',
  'del',
  'permiso'],
 ['centro',
  'integra',
  'monika',
  'ayuda',
  'mujer',
  'inmigrant',
  'desempleada',
  'encontr',
  'trabajo'],
 ['cuánto', 'año', 'le', 'llevará', 'devolv', 'préstamo'],
 ['ciudad',
  'má',
  'grand',
  'esta',
  'enfermedad',
  'se',
  'tratan',
  'separ',
  'policlínica',
  'para',
  'enfermedad',
  'transmisión',
  'sexual',
  'sukupuolitautien',
  'poliklinikka'],
 ['si',
  'se',
  'aprueba',
  'su',
  'solicitud',
  'asilo',
  'o',
  'recibe',
  'permiso',
  'residencia',
  'alguna',
  'otra',
  'razón',
  'podrá',
  'permanec',
  'finlandia'],
 ['otra',
  'alternativa',
  '',
  'traduzca',
  'documento',
  'traductor',
  'jur',
  'autoriz',
  'algún',
  'est',
  'miembro',
  'ue'],
 ['aunque',
  'haya',
  'lleg',
  'finlandia',

La funcion "Crear_vocabulario" crea el vocabulario a partir del corpus asignando un identificador unico, recorriendo cada palabra del corpus.

In [ ]:
# Función para crear un vocabulario con identificadores únicos para cada palabra
def crear_vocabulario(corpus):
  vocabulario = {}
  identificador = 0
  for linea in corpus:
    for palabra in linea:
      if palabra not in vocabulario:
        vocabulario[palabra] = identificador
        identificador += 1
  return vocabulario

vocabulario = crear_vocabulario(corpus_2)
vocabulario

{'ejemplo': 0,
 'matrimonio': 1,
 'haya': 2,
 'dur': 3,
 'meno': 4,
 'cinco': 5,
 'año': 6,
 'bien': 7,
 'no': 8,
 'se': 9,
 'reparten': 10,
 'necesaria': 11,
 'part': 12,
 'igual': 13,
 'deberá': 14,
 'abon': 15,
 'pago': 16,
 'momento': 17,
 'solicit': 18,
 'prórroga': 19,
 'del': 20,
 'permiso': 21,
 'centro': 22,
 'integra': 23,
 'monika': 24,
 'ayuda': 25,
 'mujer': 26,
 'inmigrant': 27,
 'desempleada': 28,
 'encontr': 29,
 'trabajo': 30,
 'cuánto': 31,
 'le': 32,
 'llevará': 33,
 'devolv': 34,
 'préstamo': 35,
 'ciudad': 36,
 'má': 37,
 'grand': 38,
 'esta': 39,
 'enfermedad': 40,
 'tratan': 41,
 'separ': 42,
 'policlínica': 43,
 'para': 44,
 'transmisión': 45,
 'sexual': 46,
 'sukupuolitautien': 47,
 'poliklinikka': 48,
 'si': 49,
 'aprueba': 50,
 'su': 51,
 'solicitud': 52,
 'asilo': 53,
 'o': 54,
 'recibe': 55,
 'residencia': 56,
 'alguna': 57,
 'otra': 58,
 'razón': 59,
 'podrá': 60,
 'permanec': 61,
 'finlandia': 62,
 'alternativa': 63,
 '': 64,
 'traduzca': 65,
 'documento'

#Brown Clustering

Este código implementa un modelo simple de Brown Clustering, un algoritmo de agrupamiento jerárquico utilizado en procesamiento de lenguaje natural para agrupar palabras según su contexto en un corpus. El objetivo del algoritmo es agrupar palabras que aparecen en contextos similares, formando clusters de palabras semánticamente relacionados. A continuación, se explica cada parte del código y el flujo general del algoritmo.

Empieza con la funcion inicializar_clusteres, cuyo parametro de entrada es el corpus y asigna un cluster único a cada palabra en el corpus, iniciando cada palabra en su propio cluster, cada cluster es un diccionario donde cada clave es un cluster_id y el valor es un conjunto de palabras.

Luego esta la funcion calcular_frecuencias_bigramas donde se calcula las frecuencias de unigramas (palabras individuales) y bigramas (pares de palabras consecutivas) y retorna un diccionario con los bigramas, un diccionario con los unigramas y el total de palabras en el corpus.

Tiene la funcion calcular_probabilidades, la cual sirve para calcular las probabilidades de clusters (P(c)) y probabilidades conjuntas de clusters (P(c_i, c_j)) basadas en las frecuencias de unigramas y bigramas.

Sigue la funcion calcular_delta_I que es sobre el calculo de la disminucion de informacion mutua, para ello  recibe 2 clusteres con sus respectivas probabilidades y hace la respectiva operacion para calcular la diferencia de informoracion que será un insumo para decidir si los clusteres se fusionan

Luego esta la funcion "fusion_clusteres" que ejecuta la fusion de clusteres, para ello recibe todos los clusteres junto a las probabilidades y va recorriendo cluster por cluster y calcula el delta con la funcion anterior para determinar si es un par que se debe fusionar o no, luego el nuevo cluster se hace mediante la unicion de los dos mejores cliusteres, es decir que tengan la mayor similitud o menor disminucion en la informacion muttua, luego le reasigna el nuevo identificador al cluster y actualiza el cluster

las funciones "reasignar_ids_consecutivos" y "actualizar_mapeo_palabras" son funciones auxiliares que ayudan a reasignar los ids a los clisteres comenzando de 0 y tambien el maping de palabras por cluster

Finalmente esta la funcion principal brown_clustering que inicia las los clusteres y probabilidades para luego entrar al bucle de fusiones de clustering hasta que se llega a la cantidad e clusters que se quiere y retorna los clusteres

In [ ]:
#brown_clustering simple
from collections import defaultdict, Counter
import numpy as np

# Inicialización de clusters: cada palabra en el corpus se asigna a un cluster único
def inicializar_clusters(corpus):
  clusters = {}
  palabra_a_cluster = {}
  cluster_id = 0
  for linea in corpus:
    for palabra in linea:
      if palabra not in palabra_a_cluster:
        clusters[cluster_id] = {palabra}
        palabra_a_cluster[palabra] = cluster_id
        cluster_id += 1
  return clusters, palabra_a_cluster

# Cálculo de frecuencias de unigramas y bigramas en el corpus
def calcular_frecuencias_bigramas(corpus):
  bigramas = Counter()
  unigramas = Counter()
  for linea in corpus:
    for i in range(len(linea) - 1):
      bigramas[(linea[i], linea[i+1])] += 1
      unigramas[linea[i]] += 1
    if linea:
      unigramas[linea[-1]] += 1
  total_unigramas = sum(unigramas.values())
  return bigramas, unigramas, total_unigramas

# Cálculo de probabilidades: P(c), P(c_i, c_j)
def calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas):
  P_c = {}
  P_c_c = defaultdict(lambda: defaultdict(float))
  for cluster_id, palabras in clusters.items():
    # Calcular la probabilidad de cada cluster en función de los unigramas
    P_c[cluster_id] = sum(unigramas[palabra] for palabra in palabras) / total_unigramas
  for (palabra1, palabra2), count in bigramas.items():
    # Calcular la probabilidad conjunta entre clusters basada en bigramas
    cluster1 = palabra_a_cluster[palabra1]
    cluster2 = palabra_a_cluster[palabra2]
    P_c_c[cluster1][cluster2] += count / total_unigramas
  return P_c, P_c_c

# Cálculo de la disminución en la información mutua (I) si se fusionan dos clusters
def calcular_delta_I(cluster_id1, cluster_id2, P_c, P_c_c):
  # Retorna un valor alto si alguno de los clusters no existe en P_c (para evitar fusionar clusters inexistentes)
  if cluster_id1 not in P_c or cluster_id2 not in P_c:
    return float('inf')

  # Información mutua antes de la fusión
  I_antes = P_c_c[cluster_id1][cluster_id2] * (
      np.log(P_c_c[cluster_id1][cluster_id2] / (P_c[cluster_id1] * P_c[cluster_id2]))
      if P_c_c[cluster_id1][cluster_id2] > 0 else 0)

  # Probabilidad de nuevo cluster después de la fusión
  P_c_nuevo = P_c[cluster_id1] + P_c[cluster_id2]

  # Información mutua después de la fusión
  I_despues = P_c_nuevo * (
      np.log(P_c_nuevo / (P_c[cluster_id1] + P_c[cluster_id2])) if P_c_nuevo > 0 else 0)

  delta_I = I_despues - I_antes
  return delta_I

# Fusión de clusters sin optimización: busca el par con menor I exhaustivamente
def fusionar_clusters(clusters, palabra_a_cluster, P_c, P_c_c):
  # Buscar el par de clusters con el menor ΔI
  mejor_delta_I = float('inf')
  mejor_par = None
  for c1 in clusters:
    for c2 in clusters:
      if c1 < c2:
        delta_I = calcular_delta_I(c1, c2, P_c, P_c_c)
        if delta_I < mejor_delta_I:
          mejor_delta_I = delta_I
          mejor_par = (c1, c2)

  # Realizar la fusión para el par óptimo encontrado
  if mejor_par is not None:
    c1, c2 = mejor_par
    nuevo_cluster = clusters[c1].union(clusters[c2])
    nuevo_id = min(c1, c2)
    clusters[nuevo_id] = nuevo_cluster
    del clusters[max(c1, c2)]

    # Actualizar el mapeo de palabras al nuevo ID de cluster
    for palabra in nuevo_cluster:
      palabra_a_cluster[palabra] = nuevo_id

  return clusters, palabra_a_cluster

# Reasigna los IDs de los clusters de forma consecutiva a partir de 0
def reasignar_ids_consecutivos(clusters):
  nuevo_clusters = {}
  id_map = {}
  nuevo_id = 0
  for old_id in clusters:
    nuevo_clusters[nuevo_id] = clusters[old_id]
    id_map[old_id] = nuevo_id
    nuevo_id += 1
  return nuevo_clusters, id_map

# Actualiza el mapeo de palabras a clusters después de reasignar los IDs
def actualizar_mapeo_palabras(palabra_a_cluster, id_map):
  for palabra, old_id in palabra_a_cluster.items():
    if old_id in id_map:
      palabra_a_cluster[palabra] = id_map[old_id]
  return palabra_a_cluster

# Función principal del algoritmo de Brown Clustering (sin mejoras)
def brown_clustering(corpus, num_clusters):
  # Inicialización de clusters individuales
  clusters, palabra_a_cluster = inicializar_clusters(corpus)
  # Cálculo de frecuencias y probabilidades iniciales
  bigramas, unigramas, total_unigramas = calcular_frecuencias_bigramas(corpus)
  # Probabilidades iniciales de clusters y transiciones
  P_c, P_c_c = calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas)

  # Iterar
  while len(clusters) > num_clusters:
    P_c, P_c_c = calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas)
    clusters, palabra_a_cluster = fusionar_clusters(clusters, palabra_a_cluster, P_c, P_c_c)

  # Reasignar IDs de los clusters de forma consecutiva y actualizar el mapeo de palabras
  clusters, id_map = reasignar_ids_consecutivos(clusters)
  palabra_a_cluster = actualizar_mapeo_palabras(palabra_a_cluster, id_map)

  return clusters


In [ ]:
import time
# segun el problema debe ser 100 clusteres de ejemplo
num_clusters = 100

# Medir el tiempo de ejecución de la versión sin mejoras
start_time = time.time()
clusters_resultantes = brown_clustering(corpus_2, num_clusters)
end_time = time.time()

for cluster_id, palabras in clusters_resultantes.items():
    print(f"Cluster {cluster_id}: {palabras}")

print(f"Tiempo de ejecución sin mejoras: {end_time - start_time} segundos")

Cluster 0: {'lituano', 'letón', 'vocabulario', 'despué', 'acordada', 'linkki', 'vínculo', 'estén', 'ammatillinen', 'ayudarán', 'prest', 'lug', 'termin', 'estudiant', 'empresario', 'mudan', 'grati', 'justificant', 'cumpla', 'explora', 'transport', 'entierro', 'influ', 'cruz', 'koulumatkatuki', 'aparato', 'asiakkaankatu', 'ocuparse', 'comida', 'turco', 'asist', 'búlgaro', 'conclu', 'allí', 'hasta', 'estudi', 'farmacia', 'cuidado', 'adolescent', 'diurno', 'italiano', 'refugiadosfiné', 'kielotie', 'debe', 'estonio', 'deberán', 'menor', 'kr', 'español', 'linkkikela', 'género', 'comienza', 'planta', 'transcurrido', 'discapacitado', 'hoitoraha', 'haya', 'suministr', 'recurso', 'edad', 'cualificacion', 'fianza', 'tfno', 'alejamiento', 'ejerc', 'totalidad', 'extranjerosinglé', 'somalí', 'infantil', 'prórroga', 'croata', 'posterior', 'discapacitadosfiné', 'deseen', 'asunto', 'calcul', 'dixi', 'tukinainen', 'garantía', 'anex', 'ehkäisyneuvola', 'pod', 'húngaro', 'francé', 'albertinkatu', 'yth', '

se aprecia cada cluster asi como las palbras que lo componen y ademas se ve el tiempo de ejecucion que es de 4395 segundos que es aproxiamadamente 1h con 13 min, , lo cual indica que el algoritmo es intensivo en cómputo. Esto se debe a la naturaleza exhaustiva del Brown Clustering en la búsqueda de fusiones óptimas y al cálculo repetido de información mutua.

Algunos clusters agrupan un gran número de palabras (por ejemplo, Cluster 0 o Cluster 13), mientras que otros tienen solo unas pocas palabras o incluso solo una palabra. Esto es común en algoritmos de clustering jerárquico, ya que los clusters con alta similitud tienden a absorber más palabras durante el proceso de fusión, mientras que los términos más específicos o infrecuentes quedan en clusters pequeños.

#brown clustering con mejoras
Este código implementa una versión mejorada del algoritmo de Brown Clustering. las mejoras se basan en el uso de las siguientes herramientas:

- Estructura de Datos: Cola de Prioridad, la cual es ideal para almacenar y gestionar los pares de clusters junto con sus valores de delta_I, que representa la disminución en la información mutua si esos clusters se fusionan.
Con una cola de prioridad, siempre podemos acceder rápidamente al par de clusters con el menor delta_I, que es el par óptimo para fusionar en cada iteración del algoritmo.

Cómo la implementamos:
Utilizamos la biblioteca estándar de Python heapq para implementar esta cola de prioridad. heapq organiza los elementos en un montículo binario (o heap), donde el elemento de menor valor se encuentra en la cima.

Esta estructura de datos nos permite optimizar la búsqueda del par de clusters a fusionar, reduciendo el tiempo de procesamiento del algoritmo en cada iteración al evitar tener que revisar todos los pares de clusters cada vez.

- Técnica de Poda: Actualización Selectiva, para que en lugar de recalcular delta_I para todos los pares de clusters después de cada fusión, solo recalculamos los pares que involucran el nuevo cluster fusionado.
Esto reduce el número de operaciones necesarias, especialmente en las primeras etapas del algoritmo, cuando el número de clusters es alto.

Esta técnica de poda reduce significativamente el número de veces que calculamos delta_I en cada iteración, ya que limitamos el cálculo a solo los nuevos pares que involucran el cluster recién fusionado.
En un escenario sin poda, recalculamos delta_I para todos los pares de clusters en cada iteración, lo cual sería ineficiente.

Resumen de Eficiencia:
- Cola de Prioridad (implementada con heapq): Permite obtener el par de clusters con el menor delta_I en O(logn), lo cual es más eficiente que revisar todos los pares.

- Poda de Actualización Selectiva: Reduce el número de cálculos de delta_I
ΔI en cada iteración, actualizando solo los pares necesarios, lo cual mejora el rendimiento general al reducir operaciones innecesarias.


El flujo de trabajo es parecido al codigo anterior per se añade el metodo "inicializar_cola_prioridad" que organiza los pares en orden ascendente segun su delta_I, lo cual permite extraer el par con la menor delta_I en cada iteración, optimizando la selección del par de clusters a fusionar.

Se modifica la funcion "fusionar_clusters", añadiendo la cola de prioridad como metodo  para pode seleccionar los clusteres ademas de una condicion adicional para evitar la funcion desproporcionada.

In [ ]:
#brown_clustering con mejoras

from collections import defaultdict, Counter
import numpy as np
import heapq

# Inicialización de clusters: cada palabra en el corpus se asigna a un cluster único
def inicializar_clusters(corpus):
  clusters = {}
  palabra_a_cluster = {}
  cluster_id = 0
  for linea in corpus:
    for palabra in linea:
      if palabra not in palabra_a_cluster:
        # Asignar cada palabra a su propio cluster
        clusters[cluster_id] = {palabra}
        palabra_a_cluster[palabra] = cluster_id
        cluster_id += 1
  return clusters, palabra_a_cluster

# Cálculo de frecuencias de unigramas y bigramas en el corpus
def calcular_frecuencias_bigramas(corpus):
  bigramas = Counter()
  unigramas = Counter()
  for linea in corpus:
    for i in range(len(linea) - 1):
      bigramas[(linea[i], linea[i+1])] += 1
      unigramas[linea[i]] += 1
    if linea:
      unigramas[linea[-1]] += 1  # Contabilizar la última palabra de la línea
  total_unigramas = sum(unigramas.values())
  return bigramas, unigramas, total_unigramas

# Cálculo de probabilidades: P(c), P(c_i, c_j)
def calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas):
  P_c = {}
  P_c_c = defaultdict(lambda: defaultdict(float))
  for cluster_id, palabras in clusters.items():
    # Calcular la probabilidad de cada cluster en función de los unigramas
    P_c[cluster_id] = sum(unigramas[palabra] for palabra in palabras) / total_unigramas
  for (palabra1, palabra2), count in bigramas.items():
    # Calcular la probabilidad conjunta entre clusters basada en bigramas
    cluster1 = palabra_a_cluster[palabra1]
    cluster2 = palabra_a_cluster[palabra2]
    P_c_c[cluster1][cluster2] += count / total_unigramas
  return P_c, P_c_c

# Cálculo de la disminución en la información mutua (I) si se fusionan dos clusters
def calcular_delta_I(cluster_id1, cluster_id2, P_c, P_c_c):
  # Retorna un valor alto si alguno de los clusters no existe en P_c (para evitar fusionar clusters inexistentes)
  if cluster_id1 not in P_c or cluster_id2 not in P_c:
    return float('inf')

  # Información mutua antes de la fusión
  I_antes = P_c_c[cluster_id1][cluster_id2] * (
      np.log(P_c_c[cluster_id1][cluster_id2] / (P_c[cluster_id1] * P_c[cluster_id2]))
      if P_c_c[cluster_id1][cluster_id2] > 0 else 0)

  # Probabilidad de nuevo cluster después de la fusión
  P_c_nuevo = P_c[cluster_id1] + P_c[cluster_id2]

  # Información mutua después de la fusión
  I_despues = P_c_nuevo * (
      np.log(P_c_nuevo / (P_c[cluster_id1] + P_c[cluster_id2])) if P_c_nuevo > 0 else 0)

  delta_I = I_despues - I_antes
  return delta_I

# Inicialización de una cola de prioridad con todos los pares de clusters y sus ΔI calculados
def inicializar_cola_prioridad(clusters, P_c, P_c_c):
  cola = []
  for cluster1 in clusters:
    for cluster2 in clusters:
      if cluster1 < cluster2:
        delta_I = calcular_delta_I(cluster1, cluster2, P_c, P_c_c)
        heapq.heappush(cola, (delta_I, cluster1, cluster2))
  return cola

# Fusión de clusters: fusiona el par de clusters con menor I usando la cola de prioridad
def fusionar_clusters(clusters, palabra_a_cluster, P_c, P_c_c, cola):
  while cola:
    delta_I, c1, c2 = heapq.heappop(cola)

    # Verificar que ambos clusters aún existan antes de acceder a ellos
    if c1 not in clusters or c2 not in clusters:
      continue

    # Condición adicional: evitar fusiones que creen clusters desproporcionadamente grandes
    if len(clusters[c1]) + len(clusters[c2]) > len(palabra_a_cluster) / 100 * 2:
      continue

    # Realizar la fusión de los clusters si ambos aún existen
    nuevo_cluster = clusters[c1].union(clusters[c2])
    nuevo_id = min(c1, c2)
    clusters[nuevo_id] = nuevo_cluster
    del clusters[max(c1, c2)]

    # Actualizar el mapeo de palabras al nuevo ID de cluster
    for palabra in nuevo_cluster:
      palabra_a_cluster[palabra] = nuevo_id

    # Actualizar la cola de prioridad con nuevos pares para el cluster fusionado
    for cluster in list(clusters.keys()):
      if cluster != nuevo_id:
        delta_I_nuevo = calcular_delta_I(nuevo_id, cluster, P_c, P_c_c)
        heapq.heappush(cola, (delta_I_nuevo, nuevo_id, cluster))
    break  # Realizar solo una fusión por iteración

  return clusters, palabra_a_cluster

# Reasigna los IDs de los clusters de forma consecutiva a partir de 0
def reasignar_ids_consecutivos(clusters):
  nuevo_clusters = {}
  id_map = {}
  nuevo_id = 0
  for old_id in clusters:
    nuevo_clusters[nuevo_id] = clusters[old_id]
    id_map[old_id] = nuevo_id
    nuevo_id += 1
  return nuevo_clusters, id_map

# Actualiza el mapeo de palabras a clusters después de reasignar los IDs
def actualizar_mapeo_palabras(palabra_a_cluster, id_map):
  for palabra, old_id in palabra_a_cluster.items():
    if old_id in id_map:
      palabra_a_cluster[palabra] = id_map[old_id]
  return palabra_a_cluster

# Función principal del algoritmo de Brown Clustering
def brown_clustering(corpus, num_clusters):
  # Inicialización de clusters individuales
  clusters, palabra_a_cluster = inicializar_clusters(corpus)

  # Cálculo de frecuencias y probabilidades iniciales
  bigramas, unigramas, total_unigramas = calcular_frecuencias_bigramas(corpus)
  P_c, P_c_c = calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas)
   # Inicializar cola de prioridad con pares de clusters
  cola = inicializar_cola_prioridad(clusters, P_c, P_c_c)

  while len(clusters) > num_clusters:
    P_c, P_c_c = calcular_probabilidades(clusters, palabra_a_cluster, bigramas, unigramas, total_unigramas)
    clusters, palabra_a_cluster = fusionar_clusters(clusters, palabra_a_cluster, P_c, P_c_c, cola)

  # Reasignar IDs de los clusters de forma consecutiva y actualizar el mapeo de palabras
  clusters, id_map = reasignar_ids_consecutivos(clusters)
  palabra_a_cluster = actualizar_mapeo_palabras(palabra_a_cluster, id_map)

  return clusters

In [ ]:
import time
# segun el problema debe ser 100 clusteres de ejemplo
num_clusters = 100

# Medir el tiempo de ejecución de la versión con mejoras
start_time = time.time()

# Llamada a la función brown_clustering
clusters_resultantes= brown_clustering(corpus_2, num_clusters)
end_time = time.time()

# Iterar sobre los clusters resultantes
for cluster_id, palabras in clusters_resultantes.items():
  print(f"Cluster {cluster_id}: {palabras}")

print(f"Tiempo de ejecución con mejoras: {end_time - start_time} segundos")

Cluster 0: {'supermercado', 'superior', 'do', 'aten', 'público', 'erityisammattitutkinto', 'médico', 'privadofiné', 'centro', 'sea', 'especial', 'salud', 'meno', 'odontológica', 'respecto', 'asesoramiento', 'año', 'como', 'servicio', 'kioski', 'librefiné', 'candidato', 'cinco', 'emple', 'así', 'caso', 'protec', 'infancia', 'mental', 'este', 'alguno', 'import', 'farmacia', 'preescol', 'r', 'al', 'jurídico', 'edad', 'enseñanza', 'aire', 'propietario', 'incremento', 'ejemplo'}
Cluster 1: {'kierraty', 'info', 'contrato', 'contra', 'oficina', 'su', 'maahanmuuttovirasto', 'registro', 'dependerá', 'inicio', 'web', 'schengen', 'propia', 'recurso', 'juvenil', 'patent', 'motiva', 'present', 'edificio', 'hijo', 'portal', 'te', 'página', 'residencia', 'matrimonio', 'del', 'área', 'menor', 'travé', 'municipio', 'cual', 'solicitud', 'casa', 'prórroga', 'permiso', 'educa', 'metropolitana', 'infantil', 'emplea', 'migracion', 'básica', 'infofinland', 'arrendador'}
Cluster 2: {'aunque', 'construida', 's

Los clusters muestran una variedad de temas, que pueden incluir educación, administración, tecnología, servicios, y otros. Esto sugiere que el corpus de texto es diverso y abarca múltiples áreas temáticas.

Ademas el tiempo de ejecución mejorado fue de 44.5 segundos, comparado con los 4395 segundos (~1 hora y 13 minutos) de la versión sin mejoras. Esto demuestra que la inclusión de la cola de prioridad y las optimizaciones en el código han tenido un impacto significativo en la eficiencia del algoritmo.

Existen varios clusters que contienen solo una palabra, lo que indica que esas palabras probablemente no compartieron suficiente contexto con otras en el corpus. Ejemplos:

#LSA

Este código implementa un modelo de Análisis Semántico Latente (Latent Semantic Analysis, LSA) usando una matriz de TF-IDF y una descomposición en valores singulares (SVD). El objetivo del LSA es reducir la dimensionalidad del espacio vectorial y descubrir relaciones semánticas entre términos y documentos.

- Paso 1: Construcción de la Matriz Término-Documento con TF-IDF
La función construir_matriz_termino_documento_tfidf construye una matriz donde cada fila representa un término (palabra), cada columna representa un documento, y los valores son los pesos TF-IDF de los términos en los documentos.

Terminos:
1.   TF (Frecuencia del Término): Proporción de la frecuencia de la palabra en el documento.
2. IDF (Frecuencia Inversa de Documento): Inversa de la frecuencia de documentos que contienen el término, penalizando términos comunes en todos los documentos.
3. Matriz de Salida X: Una matriz de tamaño m x n donde m es el número de términos y n es el número de documentos.

- Paso 2: La función aplicar_svd realiza la Descomposición en Valores Singulares (SVD) sobre la matriz de TF-IDF, dividiéndola en tres matrices: U, Sigma, y Vt.

Variables:
1. U: Matriz de términos singulares (cada columna es un vector singular asociado a un término).
2. Sigma: Vector de valores singulares que representan la "importancia" de cada dimensión en el espacio.
3. Vt: Matriz de documentos singulares (cada fila es un vector singular asociado a un documento).

- Paso 3: Paso 3: Reducción de Dimensionalidad y Obtención de X_k
La función obtener_matriz_reducida reduce la dimensionalidad de X seleccionando los k valores singulares más grandes y sus correspondientes vectores singulares en U y Vt.

Variables:

1. U_k: Selección de los primeros k vectores de U.
2. Sigma_k: Matriz diagonal construida con los primeros k valores singulares.
3. Vt_k: Selección de las primeras k filas de Vt.
4. X_k: Aproximación de la matriz X en un espacio de menor dimensión. Esta matriz reduce el "ruido" o información menos relevante en la matriz original.

- Paso 4: Proyección en el Espacio Reducido Termino - documentos
La función proyectar_espacio_reducido representa los términos y documentos en el espacio reducido, obteniendo una aproximación de la matriz término-documento

In [ ]:
import math

# Paso 1: Construcción de la matriz término-documento con TF-IDF
def construir_matriz_termino_documento_tfidf(corpus, vocabulario):
  """
  Construye una matriz término-documento donde las filas representan términos y
  las columnas representan documentos. Los valores son pesos TF-IDF de términos en los documentos.
  """
  m = len(vocabulario)  # Número de términos en el vocabulario
  n = len(corpus)       # Número de documentos en el corpus
  X = np.zeros((m, n))  # Matriz término-documento inicializada con ceros

  # Calcular la frecuencia de documentos para cada término
  doc_freq = {palabra: 0 for palabra in vocabulario}
  for documento in corpus:
    unique_words = set(documento)  # Contar cada término una sola vez por documento
    for palabra in unique_words:
      if palabra in doc_freq:
        doc_freq[palabra] += 1

  # Llenar la matriz término-documento con pesos TF-IDF
  for j, documento in enumerate(corpus):
    term_freq = {}
    for palabra in documento:
      if palabra in vocabulario:
        term_freq[palabra] = term_freq.get(palabra, 0) + 1  # TF: Contador de frecuencia para el término

    for palabra, freq in term_freq.items():
      i = vocabulario[palabra]  # Índice del término en el vocabulario
      tf = freq / len(documento)  # TF: Frecuencia del término en el documento
      idf = math.log(n / (doc_freq[palabra] + 1))  # IDF: Logaritmo de N / número de documentos que contienen el término
      X[i, j] = tf * idf  # Peso TF-IDF

  return X

# Paso 2: Implementación de SVD
def aplicar_svd(X):
  """
  Realiza la descomposición en valores singulares (SVD) en la matriz término-documento X,
  obteniendo las matrices U, Sigma y Vt.
  """
  U, Sigma, Vt = np.linalg.svd(X, full_matrices=False)
  return U, Sigma, Vt

# Paso 3: Reducción de dimensionalidad y obtención de X_k
def obtener_matriz_reducida(U, Sigma, Vt, k):
  """
  Selecciona los k valores singulares más grandes y calcula la aproximación de rango k de X,
  representada por X_k = U_k * Sigma_k * Vt_k.
  """
  U_k = U[:, :k]               # Seleccionar los primeros k vectores singulares de U
  Sigma_k = np.diag(Sigma[:k])  # Construir matriz diagonal de los primeros k valores singulares
  Vt_k = Vt[:k, :]              # Seleccionar los primeros k vectores singulares de Vt
  X_k = U_k @ Sigma_k @ Vt_k    # Calcular X_k
  return U_k, Sigma_k, Vt_k, X_k

# Paso 4: Proyección en el espacio reducido
def proyectar_espacio_reducido(U_k, Sigma_k, Vt_k):
  """
  Representa términos y documentos en el espacio reducido utilizando las matrices resultantes
  de la reducción de dimensionalidad.
  """
  X_k = U_k @ Sigma_k @ Vt_k  # Aproximación de la matriz término-documento en el espacio reducido
  return X_k

In [ ]:
# Paso 1: Construcción de la matriz término-documento con TF-IDF
X = construir_matriz_termino_documento_tfidf(corpus_2, vocabulario)
print("=== Matriz Término-Documento (TF-IDF) ===")
print(np.round(X, 6))

# Paso 2: Implementación de SVD
U, Sigma, Vt = aplicar_svd(X)
print("\n=== Matriz U (Términos) ===")
print(np.round(U, 6))
print("\n=== Valores Singulares (Sigma) ===")
print(np.round(Sigma, 6))
print("\n=== Matriz Vt (Documentos) ===")
print(np.round(Vt, 6))

# Paso 3: Reducción de dimensionalidad y cálculo de X_k
k = 4 # Número de dimensiones en el espacio reducido
U_k, Sigma_k, Vt_k, X_k = obtener_matriz_reducida(U, Sigma, Vt, k)
print("\n=== Aproximación de Rango k de la Matriz Término-Documento (X_k) ===")
print(np.round(X_k, 6))

# Paso 4: Proyección en el espacio reducido

representacion_terminos = U_k @ Sigma_k  # Representación de términos en el espacio reducido
representacion_documentos = Vt_k.T @ Sigma_k  # Representación de documentos en el espacio reducido

print("\n=== Representación de Términos en el Espacio Reducido ===")
print(np.round(representacion_terminos, 6))

print("\n=== Representación de Documentos en el Espacio Reducido ===")
print(np.round(representacion_documentos, 6))

X_k_proyectado = proyectar_espacio_reducido(U_k, Sigma_k, Vt_k)
print("\n=== Proyección Terminos - Documentos en el Espacio Reducido ===")
print(np.round(X_k_proyectado, 5))


=== Matriz Término-Documento (TF-IDF) ===
[[0.235488 0.       0.       ... 0.       0.       0.      ]
 [0.365428 0.       0.       ... 0.       0.       0.      ]
 [0.3102   0.       0.       ... 0.       0.       0.      ]
 ...
 [0.       0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.       ... 0.       0.776826 0.      ]]

=== Matriz U (Términos) ===
[[ 0.        0.       -0.       ... -0.036792 -0.020492 -0.005215]
 [-0.        0.        0.       ... -0.057311 -0.037089  0.006004]
 [-0.        0.        0.       ...  0.017166 -0.040251 -0.01878 ]
 ...
 [ 0.        0.        0.       ... -0.010444 -0.01224  -0.004254]
 [ 0.        0.        0.       ... -0.010444 -0.01224  -0.004254]
 [ 0.       -0.        0.       ...  0.004243  0.021995 -0.001525]]

=== Valores Singulares (Sigma) ===
[13.815511 12.153989 11.439714 11.439714 11.439714 10.055577  9.563451
  8.532661  8.215369  8.215369  7.000564  

Se hace las impresiones de los resultados obtenidos en cada paso

1. Matriz Termino Documento: La matriz TF-IDF (X) contiene los pesos de cada término en cada documento, basados en la frecuencia de ocurrencia del término en el documento y la frecuencia inversa en el corpus.

2. matriz de terminso: La matriz U representa los vectores singulares de los términos después de aplicar la descomposición en valores singulares (SVD). Cada columna en U describe un término en el espacio de características, mostrando su importancia relativa en cada componente principal.

3. valores singulares: Los valores más altos indican las dimensiones más significativas, mientras que los valores más bajos se pueden descartar en la reducción de dimensionalidad.

4. vectores singulares de documentos: Esta aproximación reduce el "ruido" y conserva las relaciones semánticas principales entre términos y documentos, permitiendo trabajar con una representación más compacta.

5. Rango k de ka matriz termino - documento (Aproximación de la matriz X en un espacio de menor dimensión. Este resultado muestra cómo los términos y documentos se representan en un espacio reducido, reteniendo la mayoría de las características relevantes y eliminando el "ruido".)

6. Representacion de Terminos: La representación de términos muestra cómo cada término se sitúa en el espacio reducido. Esto es útil para identificar la relación semántica entre palabras, ya que los términos cercanos en este espacio pueden estar relacionados en contexto.

7. representacion de Documentos: La representación de documentos muestra la posición de cada documento en el espacio reducido. Los documentos cercanos entre sí en este espacio tienen similitudes semánticas, y esta representación es útil para comparar documentos o realizar recomendaciones.

8.  Matriz Término-Documento en el espacio reducido: La proyección en el espacio reducido (X_k_proyectado) es la representación final de la matriz término-documento, donde tanto términos como documentos están en un espacio de menor dimensionalidad. Esto facilita el análisis de similitudes y relaciones entre términos y documentos con menor complejidad.

#LSA con matriz termino-documento

Este código implementa un modelo de Análisis Semántico Latente (LSA) mejorado con dos optimizaciones importantes: el uso de una matriz término-documento dispersa y el método de potencias para calcular el valor singular y vector singular dominante de la matriz. Estas optimizaciones están diseñadas para trabajar con corpus de gran tamaño, donde una matriz densa y un cálculo SVD completo pueden ser computacionalmente costosos.

- Paso 1:  Construcción de la Matriz Término-Documento Dispersa con TF-IDF
La función construir_matriz_termino_documento_tfidf_disperso crea una matriz dispersa de TF-IDF usando la biblioteca scipy.sparse. Una matriz dispersa almacena solo los valores diferentes de cero, lo cual es mucho más eficiente en términos de memoria y cómputo para grandes corpus.

Parámetros:
1. corpus: Lista de documentos, donde cada documento es una lista de palabras.
2. vocabulario: Diccionario que mapea cada palabra a un índice único.
3. Salida_1 X_disperso, una matriz dispersa en formato COO (scipy.sparse.coo_matrix) que contiene los valores TF-IDF de los términos en los documentos.

- Paso 2: Método de Potencias para el Cálculo del Valor y Vector Singular Dominante
La función metodo_potencias implementa el método de potencias para encontrar el valor singular dominante y su correspondiente vector singular en la matriz término-documento dispersa X. Este método es iterativo y es particularmente útil para matrices grandes donde el cálculo completo de SVD es ineficiente.

Parámetros:
1. X: Matriz término-documento dispersa.
2. num_iter: Número máximo de iteraciones para el método de potencias.
3. tol: Tolerancia para la convergencia (criterio de parada).
4. Salida_1 valor_singular: El valor singular dominante, calculado como la raíz cuadrada de b_k1_norm.
5. Salida 2 vector_singular: El vector singular dominante correspondiente



In [ ]:
#LSA con las nuevas implementaciones (potencia y  Matriz Término-Documento Dispersa)
# La implementacion tiene dificultades en el calculo de la potencia u.u

import math
from scipy.sparse import coo_matrix

# Paso 1: Construcción de la matriz término-documento dispersa con TF-IDF
def construir_matriz_termino_documento_tfidf_disperso(corpus, vocabulario):
  m = len(vocabulario)
  n = len(corpus)
  doc_freq = {palabra: 0 for palabra in vocabulario}

  for documento in corpus:
    unique_words = set(documento)
    for palabra in unique_words:
      if palabra in doc_freq:
        doc_freq[palabra] += 1

  row_indices = []
  col_indices = []
  data = []

  for j, documento in enumerate(corpus):
    term_freq = {}
    for palabra in documento:
      if palabra in vocabulario:
        term_freq[palabra] = term_freq.get(palabra, 0) + 1

    for palabra, freq in term_freq.items():
      i = vocabulario[palabra]
      tf = freq / len(documento)
      idf = math.log(n / (doc_freq[palabra] + 1))
      row_indices.append(i)
      col_indices.append(j)
      data.append(tf * idf)

  X_disperso = coo_matrix((data, (row_indices, col_indices)), shape=(m, n))
  return X_disperso

# Método de Potencias para el cálculo del valor y vector singular más significativo
def metodo_potencias(X, num_iter=100, tol=1e-6):
  b_k = np.random.rand(X.shape[1])
  for _ in range(num_iter):
    b_k1 = X.T @ (X @ b_k)
    b_k1_norm = np.linalg.norm(b_k1)
    b_k1 = b_k1 / b_k1_norm
    if np.linalg.norm(b_k - b_k1) < tol:
      break
    b_k = b_k1

  valor_singular = np.sqrt(b_k1_norm)
  vector_singular = b_k1
  return valor_singular, vector_singular

In [ ]:
# Paso 1: Impresión de la matriz término-documento con matriz dispersa (TF-IDF)
X_disperso = construir_matriz_termino_documento_tfidf_disperso(corpus_2, vocabulario)
X = X_disperso.toarray()  # Convertimos la matriz dispersa a formato denso para el cálculo
print("=== Matriz Término-Documento (TF-IDF) ===")
print(np.round(X, 6))

# Paso 2: Calcular el valor y vector singular más significativo usando el método de potencias
valor_singular, vector_singular = metodo_potencias(X)
print("\n=== Valor Singular más Significativo ===")
print(round(valor_singular, 6))
print("\n=== Vector Singular más Significativo ===")
print(np.round(vector_singular, 6))

# Paso 3: Representación en el espacio reducido (con el vector singular más dominante)
k = 5
U_k = vector_singular.reshape(-1, 1)  # La representación de términos
Sigma_k = np.array([[valor_singular]])  # Valor singular más grande en forma diagonal
Vt_k = U_k.T  # Representación de documentos en el espacio reducido

# Paso 4: Proyección en el espacio reducido
X_k_proyectado = U_k @ Sigma_k @ Vt_k
print("\n=== Proyección en el Espacio Reducido (Aproximación con el Valor Singular más Dominante) ===")
print(np.round(X_k_proyectado, 6))

# Representaciones en el espacio reducido
representacion_terminos = U_k @ Sigma_k  # Representación de términos en el espacio reducido
representacion_documentos = Vt_k.T @ Sigma_k  # Representación de documentos en el espacio reducido

print("\n=== Representación de Términos en el Espacio Reducido ===")
print(np.round(representacion_terminos, 6))

print("\n=== Representación de Documentos en el Espacio Reducido ===")
print(np.round(representacion_documentos, 6))

=== Matriz Término-Documento (TF-IDF) ===
[[0.235488 0.       0.       ... 0.       0.       0.      ]
 [0.365428 0.       0.       ... 0.       0.       0.      ]
 [0.3102   0.       0.       ... 0.       0.       0.      ]
 ...
 [0.       0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.       ... 0.       0.776826 0.      ]]

=== Valor Singular más Significativo ===
13.815511

=== Vector Singular más Significativo ===
[0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 3.33333e-01 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00

Las impresiones son:

1.Matriz Término-Documento (TF-IDF) : Esta matriz muestra los pesos TF-IDF de los términos en los documentos en formato denso (convertida desde la versión dispersa). Cada fila representa un término, y cada columna representa un documento. Los valores distintos de cero reflejan la importancia de cada término en un documento específico.

2. Valor Singular más Significativo: El valor singular dominante calculado es 13.815511. Este valor singular representa la dimensión de mayor importancia en la matriz TF-IDF y se usa para la reducción de dimensionalidad en el espacio semántico.

3. Vector Singular más Significativo : El vector singular dominante muestra los pesos asociados a cada término en la dimensión más importante. Este vector se utiliza para proyectar términos y documentos en el espacio reducido, capturando la estructura semántica principal de los datos.

4. Proyección en el Espacio Reducido: La matriz X_k_proyectado representa la aproximación de la matriz término-documento en un espacio reducido usando solo el valor y el vector singular dominante. La proyección en el espacio reducido ayuda a simplificar el análisis y es útil para identificar relaciones entre términos y documentos de manera más compacta.

5. Representación de Términos en el Espacio Reducido: La representación de términos en el espacio reducido muestra cómo cada término se posiciona en la dimensión más significativa del espacio semántico. Esto permite observar la relación semántica entre términos, ya que los términos con valores similares en esta dimensión pueden tener un contexto común en el corpus.

6. Representación de Documentos en el Espacio Reducido: La representación de documentos en el espacio reducido muestra la proyección de cada documento en el espacio de menor dimensión. Los documentos con valores similares en esta proyección están relacionados semánticamente, lo cual facilita la búsqueda de documentos similares o relevantes.

#Word2vec

Este código implementa un modelo de Word2Vec con dos enfoques principales: CBOW (Continuous Bag of Words) y Skip-Gram con Negative Sampling. Ambos métodos son populares para generar embeddings de palabras, que representan las palabras en un espacio vectorial donde las palabras con contextos similares se agrupan cercanamente.

1. Función generar_pares_contexto_objetivo crea pares (palabra_objetivo, contexto) a partir de una ventana de contexto especificada alrededor de cada palabra. Estos pares serán usados en los modelos CBOW y Skip-Gram para entrenar el modelo.

Parámetros de entrada:
1. corpus: Lista de documentos, donde cada documento es una lista de palabras.
2. ventana: Tamaño de la ventana de contexto (predeterminado a 2).

Parametros de Salida:
- Lista de pares (palabra_objetivo, contexto), donde el contexto es una lista de palabras cercanas a la palabra_objetivo.

2. Función inicializar_pesos crea las matrices de pesos para las capas de entrada y salida.

Parámetros de entrada:
1. vocab_size: Tamaño del vocabulario.
2. embedding_dim: Dimensión del espacio de embedding.

Parametros de Salida:
1. matriz para la capa de entrada (pesos_entrada).
2. matriz para la capa de salida (pesos_salida).

3. Funciones softmax y sigmoid
- softmax normaliza un vector para obtener probabilidades.
- sigmoid es una función de activación que se usa para el cálculo de probabilidades en el contexto de Skip-Gram con Negative Sampling.

4. Función entrenar_cbow implementa el modelo CBOW, donde el objetivo es predecir la palabra objetivo (target) usando el contexto. Para ello primero calcula el vector promedio de los embedidings del contexto, luego predice la palabra objetivo con la funcion softmax y finalmente calcula la perdida y realiza el propagation para actualizar los pesos.

5. función generar_muestras_negativas selecciona indices de palabras que no son el objetivo para usarlas en el entrenamiento del modelo Skip-Gram con Negative Sampling.

Parámetros:
- vocab_size: Tamaño del vocabulario.
- target_index: Índice de la palabra objetivo.
- num_neg_samples: Número de muestras negativas a generar.

6. función entrenar_skipgram implementa el modelo Skip-Gram, donde el objetivo es predecir el contexto dado una palabra objetivo, usando muestras negativas. Para ello calcula la prediccion del contexto usando el embedding de la palabra objetivo, luego actualiza los pesos en funcion de los contextos positivos y realiza Negative Sampling para contextos negrativos, finalmente calcula la perdida total y realiza el propagation

In [ ]:
#word2vec

# Función para definir la ventana de contexto y generar pares (contexto, objetivo) para CBOW y Skip-Gram
def generar_pares_contexto_objetivo(corpus, ventana=2):
  pares_contexto_objetivo = []
  for palabras in corpus:
    for i, palabra_objetivo in enumerate(palabras):
      ventana_inicio = max(0, i - ventana)
      ventana_fin = min(len(palabras), i + ventana + 1)
      contexto = [palabras[j] for j in range(ventana_inicio, ventana_fin) if j != i]
      pares_contexto_objetivo.append((palabra_objetivo,contexto))  # CBOW: contexto como lista, objetivo como palabra
  return pares_contexto_objetivo

# Función de inicialización de pesos
def inicializar_pesos(vocab_size, embedding_dim):
  pesos_entrada = np.random.randn(vocab_size, embedding_dim) * 0.01
  pesos_salida = np.random.randn(embedding_dim, vocab_size) * 0.01
  return pesos_entrada, pesos_salida

# Función softmax
def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)

# Función sigmoid
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# Implementación de CBOW
def entrenar_cbow(pares, vocabulario, pesos_entrada, pesos_salida, epochs=10, learning_rate=0.01):
  for epoch in range(epochs):
    loss = 0
    for objetivo, contexto in pares:
      contexto_indices = [vocabulario[palabra] for palabra in contexto if palabra in vocabulario]
      if not contexto_indices:
        continue
      h = np.mean(pesos_entrada[contexto_indices], axis=0)
      u = np.dot(h, pesos_salida)
      y_pred = softmax(u)
      target_index = vocabulario[objetivo]
      loss += -np.log(y_pred[target_index] + 1e-9)
      y_pred[target_index] -= 1
      grad_salida = np.outer(h, y_pred)
      grad_entrada = np.dot(pesos_salida, y_pred)
      pesos_salida -= learning_rate * grad_salida
      for idx in contexto_indices:
        pesos_entrada[idx] -= learning_rate * grad_entrada
    print(f"Epoca {epoch + 1}, Perdida: {loss / len(pares)}")

# Generación de muestras negativas
def generar_muestras_negativas(vocab_size, target_index, num_neg_samples):
  neg_samples = []
  while len(neg_samples) < num_neg_samples:
    muestra = random.randint(0, vocab_size - 1)
    if muestra != target_index:
      neg_samples.append(muestra)
  return neg_samples

# Implementación de Skip-Gram con Negative Sampling
def entrenar_skipgram(pares, vocabulario, pesos_entrada, pesos_salida, num_neg_samples=5, epochs=10, learning_rate=0.01):
  for epoch in range(epochs):
    loss = 0
    for objetivo, contexto in pares:
      # Paso 1: Predicción de contexto
      target_index = vocabulario[objetivo]
      contexto_indices = [vocabulario[palabra] for palabra in contexto if palabra in vocabulario]
      if not contexto_indices:
        continue

      # Embedding de la palabra objetivo
      h = pesos_entrada[target_index]

      # Paso 2: Calcular pérdida y actualizar para cada palabra de contexto positivo
      for context_index in contexto_indices:
        # Producto interno para la palabra de contexto positiva
        u = np.dot(h, pesos_salida[:, context_index])
        # Función de pérdida para pares positivos
        loss -= np.log(sigmoid(u) + 1e-9)
        # Backpropagation para actualizar pesos de la palabra de contexto positiva
        grad_salida = (1 - sigmoid(u)) * h
        grad_entrada = (1 - sigmoid(u)) * pesos_salida[:, context_index]
        pesos_salida[:, context_index] -= learning_rate * grad_salida
        pesos_entrada[target_index] -= learning_rate * grad_entrada

        # Paso 3: Negative Sampling - Actualizar pesos para muestras negativas
        neg_samples = generar_muestras_negativas(len(vocabulario), target_index, num_neg_samples)
        for neg_sample in neg_samples:
          # Producto interno para las muestras negativas
          u_neg = np.dot(h, pesos_salida[:, neg_sample])
          # Función de pérdida para pares negativos
          loss -= np.log(1 - sigmoid(u_neg) + 1e-9)
          # Backpropagation para actualizar pesos de las muestras negativas
          grad_salida_neg = -sigmoid(u_neg) * h
          grad_entrada_neg = -sigmoid(u_neg) * pesos_salida[:, neg_sample]
          pesos_salida[:, neg_sample] -= learning_rate * grad_salida_neg
          pesos_entrada[target_index] -= learning_rate * grad_entrada_neg

    print(f"Epoca {epoch + 1}, Perdida: {loss / len(pares)}")

In [ ]:
pares_contexto_objetivo = generar_pares_contexto_objetivo(corpus_2, ventana=2)
pesos_entrada, pesos_salida = inicializar_pesos(len(vocabulario), embedding_dim=10)

# Entrenar CBOW
print("Entrenando CBOW...")
entrenar_cbow(pares_contexto_objetivo, vocabulario, pesos_entrada, pesos_salida, epochs=100)

# Visualización o evaluación de embeddings
print("\n=== Embeddings de 10 Palabras Aleatorias ===")
palabras_aleatorias = random.sample(list(vocabulario.keys()), 10)
for palabra in palabras_aleatorias:
    palabra_index = vocabulario[palabra]
    print(f"Embedding para '{palabra}': {pesos_entrada[palabra_index]}")


Entrenando CBOW...
Epoca 1, Perdida: 7.611822227844279
Epoca 2, Perdida: 7.611743840004323
Epoca 3, Perdida: 7.611554595529157
Epoca 4, Perdida: 7.610999089178671
Epoca 5, Perdida: 7.609238740664557
Epoca 6, Perdida: 7.603471942511171
Epoca 7, Perdida: 7.584293165852753
Epoca 8, Perdida: 7.521006757811212
Epoca 9, Perdida: 7.365458399620583
Epoca 10, Perdida: 7.185163500274005
Epoca 11, Perdida: 7.010666302499191
Epoca 12, Perdida: 6.876876466291998
Epoca 13, Perdida: 6.770084565617013
Epoca 14, Perdida: 6.681635352453218
Epoca 15, Perdida: 6.606374261156766
Epoca 16, Perdida: 6.540189860397715
Epoca 17, Perdida: 6.479591204464477
Epoca 18, Perdida: 6.422643377224287
Epoca 19, Perdida: 6.368754202505866
Epoca 20, Perdida: 6.317485248139383
Epoca 21, Perdida: 6.268304015799664
Epoca 22, Perdida: 6.220850903093602
Epoca 23, Perdida: 6.174800918312415
Epoca 24, Perdida: 6.129743009193773
Epoca 25, Perdida: 6.085254443641525
Epoca 26, Perdida: 6.040998531494639
Epoca 27, Perdida: 5.9967840

usando el enfoque CBOW. Después de 100 épocas, se imprimieron las perdidas (loss) para cada epoca, mostrando cómo disminuye progresivamente, lo cual indica que el modelo está aprendiendo las relaciones entre palabras y contextos en el corpus.

Para evaluar el resultado del entrenamiento, se imprimieron los vectores de embeddings de 10 palabras seleccionadas aleatoriamente del vocabulario. Cada embedding es un vector de 10 dimensiones que representa la palabra en el espacio de embedding. Los valores en estos vectores representan la relación de cada palabra con su contexto, capturada durante el proceso de entrenamiento.

Los valores en el embedding de cada palabra reflejan las características semánticas aprendidas del contexto en el que aparece cada palabra en el corpus. Palabras con significados similares o que aparecen en contextos similares tienden a tener vectores de embedding con valores similares.


In [ ]:
# Reinicializar pesos para Skip-Gram
pesos_entrada, pesos_salida = inicializar_pesos(len(vocabulario), embedding_dim=10)

# Entrenar Skip-Gram
print("\nEntrenando Skip-Gram...")
entrenar_skipgram(pares_contexto_objetivo, vocabulario, pesos_entrada, pesos_salida, epochs=100)

# Visualización o evaluación de embeddings
print("\n=== Embeddings de 10 Palabras Aleatorias ===")
palabras_aleatorias = random.sample(list(vocabulario.keys()), 10)
for palabra in palabras_aleatorias:
    palabra_index = vocabulario[palabra]
    print(f"Embedding para '{palabra}': {pesos_entrada[palabra_index]}")


Entrenando Skip-Gram...
Epoca 1, Perdida: 13.843861895723373
Epoca 2, Perdida: 13.844025841469772
Epoca 3, Perdida: 13.844908282623352
Epoca 4, Perdida: 13.850330596713203
Epoca 5, Perdida: 13.88687101753076
Epoca 6, Perdida: 14.192820206888952
Epoca 7, Perdida: 23.326074861570703


<ipython-input-52-6df02063f408>:27: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoca 8, Perdida: 176.44791685447785
Epoca 9, Perdida: 350.70570626826986
Epoca 10, Perdida: 360.7318733276843
Epoca 11, Perdida: 360.687762789811
Epoca 12, Perdida: 360.69252840228404
Epoca 13, Perdida: 360.6043645715373
Epoca 14, Perdida: 360.5400288031551
Epoca 15, Perdida: 360.8617076450668
Epoca 16, Perdida: 360.67346595239303
Epoca 17, Perdida: 360.86409045130364
Epoca 18, Perdida: 360.8831529011946
Epoca 19, Perdida: 360.6162786027189
Epoca 20, Perdida: 360.91889499474036
Epoca 21, Perdida: 360.9355746383958
Epoca 22, Perdida: 360.761629783139
Epoca 23, Perdida: 360.5066695158453
Epoca 24, Perdida: 360.65440350250117
Epoca 25, Perdida: 360.7044424334661
Epoca 26, Perdida: 361.0189728566693
Epoca 27, Perdida: 360.67823156486514
Epoca 28, Perdida: 360.76639539561177
Epoca 29, Perdida: 361.0285040816151
Epoca 30, Perdida: 360.84264519517643
Epoca 31, Perdida: 360.5733880904645
Epoca 32, Perdida: 360.5281147719725
Epoca 33, Perdida: 360.8402623889407
Epoca 34, Perdida: 361.154792812

e utilizó el modelo Skip-Gram con Negative Sampling para entrenar el modelo Word2Vec en 100 épocas. Sin embargo, los resultados muestran algunos problemas con el entrenamiento, ya que los valores de pérdida se han vuelto extremadamente grandes, y los embeddings presentan valores enormes, lo que indica una posible inestabilidad numérica.

Esto puede deberse a un learning rate muy alto, que causa que los pasos de actualización en cada época sean demasiado grandes o tambien pesos con valores muy grandes.

#Glove
Este código implementa un modelo de GloVe (Global Vectors for Word Representation), que es un modelo de embeddings de palabras basado en matrices de co-ocurrencia. A diferencia de Word2Vec, que aprende embeddings optimizando las predicciones de palabras de contexto, GloVe aprende embeddings minimizando la diferencia entre el producto escalar de los vectores de palabras y el logaritmo de su frecuencia de co-ocurrencia.

Para ello se implementa lo siguiente:

1. La función construir_matriz_coocurrencia crea una matriz de co-ocurrencia que cuenta cuántas veces cada palabra aparece en el contexto de otra palabra dentro de una ventana determinada.

- Parámetros de entrada:
1. corpus: Lista de documentos, donde cada documento es una lista de palabras.
2. vocabulario: Diccionario que asigna un índice a cada palabra.
3. ventana: Tamaño de la ventana de contexto para considerar las co-ocurrencias.
- Parámetros de Salida:
1. matriz_coocurrencia, donde cada entrada indica la frecuencia de co-ocurrencia entre palabras en el contexto.

2. La función inicializar_vectores crea los vectores de embeddings y los sesgos necesarios para el modelo GloVe.

- Parametros de entrada:
1. tam_vocabulario: Número total de palabras en el vocabulario.
2. dim_embedding: Dimensión de los embeddings.
- Parametros de Salida: Cuatro matrices de parámetros (W, W_tilde, b, b_tilde) inicializadas con valores aleatorios o ceros

3. La función funcion_peso implementa una ponderación que ajusta la contribución de cada par de palabras en la función de pérdida. GloVe utiliza esta función para dar menor importancia a co-ocurrencias raras.

Parámetros de entrada:
1. X_ij: Frecuencia de co-ocurrencia entre dos palabras.
2. x_max: Umbral máximo para ajustar la ponderación.
3. alfa: Exponente que controla cómo decae el peso para co-ocurrencias menos frecuentes.
Parámetros de Salida:
1. Valor de ponderación para el par de palabras.

4. La función entrenar_glove entrena el modelo GloVe minimizando la diferencia entre el producto escalar de los embeddings y el logaritmo de la co-ocurrencia. Para ello se hace una serie de bucles anidados que para cada par de palabras en la matriz de co-ocurrencia calcula la perdida, los gradiente y actualiza los parametros y al final de cada época, se imprime la pérdida total, lo que permite monitorear el progreso del entrenamiento.


In [ ]:
# Paso 1: Construcción de la Matriz de Co-ocurrencia
def construir_matriz_coocurrencia(corpus, vocabulario, ventana=2):
  tam_vocabulario = len(vocabulario)
  matriz_coocurrencia = np.zeros((tam_vocabulario, tam_vocabulario))

  for palabras in corpus:
    for i, palabra_objetivo in enumerate(palabras):
      indice_objetivo = vocabulario[palabra_objetivo]
      inicio_ventana = max(0, i - ventana)
      fin_ventana = min(len(palabras), i + ventana + 1)
      for j in range(inicio_ventana, fin_ventana):
        if i != j:
          indice_contexto = vocabulario[palabras[j]]
          matriz_coocurrencia[indice_objetivo][indice_contexto] += 1

  return matriz_coocurrencia

# Paso 2: Inicialización de Vectores y Sesgos
def inicializar_vectores(tam_vocabulario, dim_embedding):
  W = np.random.rand(tam_vocabulario, dim_embedding) * 0.01
  W_tilde = np.random.rand(tam_vocabulario, dim_embedding) * 0.01
  b = np.zeros(tam_vocabulario)
  b_tilde = np.zeros(tam_vocabulario)
  return W, W_tilde, b, b_tilde

# Función f(X) para el cálculo de la función de costo
def funcion_peso(X_ij, x_max=100, alfa=0.75):
  if X_ij < x_max:
    return (X_ij / x_max) ** alfa
  else:
    return 1

# Paso 3: Optimización de la Función de Costo
def entrenar_glove(matriz_coocurrencia, W, W_tilde, b, b_tilde, epocas=100, tasa_aprendizaje=0.05):
  tam_vocabulario = matriz_coocurrencia.shape[0]
  for epoca in range(epocas):
    perdida_total = 0
    for i in range(tam_vocabulario):
      for j in range(tam_vocabulario):
        X_ij = matriz_coocurrencia[i][j]
        if X_ij > 0:
          # Cálculo de la función de costo
          f_X_ij = funcion_peso(X_ij)
          costo = f_X_ij * (np.dot(W[i], W_tilde[j]) + b[i] + b_tilde[j] - np.log(X_ij)) ** 2
          perdida_total += 0.5 * costo

          # Gradientes
          grad_W = f_X_ij * (np.dot(W[i], W_tilde[j]) + b[i] + b_tilde[j] - np.log(X_ij)) * W_tilde[j]
          grad_W_tilde = f_X_ij * (np.dot(W[i], W_tilde[j]) + b[i] + b_tilde[j] - np.log(X_ij)) * W[i]
          grad_b = f_X_ij * (np.dot(W[i], W_tilde[j]) + b[i] + b_tilde[j] - np.log(X_ij))
          grad_b_tilde = grad_b

          # Actualización de parámetros
          W[i] -= tasa_aprendizaje * grad_W
          W_tilde[j] -= tasa_aprendizaje * grad_W_tilde
          b[i] -= tasa_aprendizaje * grad_b
          b_tilde[j] -= tasa_aprendizaje * grad_b_tilde

    print(f"Época {epoca + 1}, Pérdida Total: {perdida_total}")

In [ ]:
matriz_coocurrencia = construir_matriz_coocurrencia(corpus_2, vocabulario, ventana=2)
dim_embedding = 10
W, W_tilde, b, b_tilde = inicializar_vectores(len(vocabulario), dim_embedding)

print("\nEntrenando GloVe...")
entrenar_glove(matriz_coocurrencia, W, W_tilde, b, b_tilde, epocas=100)

# Prueba: Mostrar los embeddings de 10 palabras aleatorias del vocabulario
print("\n=== Embeddings de 10 Palabras Aleatorias ===")
palabras_aleatorias = random.sample(list(vocabulario.keys()), 10)
for palabra in palabras_aleatorias:
    indice_palabra = vocabulario[palabra]
    embedding_palabra = W[indice_palabra]
    print(f"Embedding para '{palabra}': {embedding_palabra}")


Entrenando GloVe...
Época 1, Pérdida Total: 215.32317389627383
Época 2, Pérdida Total: 173.73001673899688
Época 3, Pérdida Total: 155.06555278038888
Época 4, Pérdida Total: 145.21890739021978
Época 5, Pérdida Total: 139.3089387299766
Época 6, Pérdida Total: 135.38564666375993
Época 7, Pérdida Total: 132.56994546899634
Época 8, Pérdida Total: 130.42588652498162
Época 9, Pérdida Total: 128.71970839043922
Época 10, Pérdida Total: 127.31708605648583
Época 11, Pérdida Total: 126.13571250056272
Época 12, Pérdida Total: 125.12205920951531
Época 13, Pérdida Total: 124.2393990822964
Época 14, Pérdida Total: 123.46133722813728
Época 15, Pérdida Total: 122.76814877099632
Época 16, Pérdida Total: 122.14460339525331
Época 17, Pérdida Total: 121.57860791398055
Época 18, Pérdida Total: 121.06031630711378
Época 19, Pérdida Total: 120.58151683435725
Época 20, Pérdida Total: 120.13518884533846
Época 21, Pérdida Total: 119.71516623775719
Época 22, Pérdida Total: 119.31586892853728
Época 23, Pérdida Tota

Los resultados muestran una disminución progresiva en la pérdida total, lo cual indica que el modelo está aprendiendo y ajustando los parámetros de manera efectiva para capturar las relaciones de co-ocurrencia en el corpus.

Se imprimieron los vectores de embeddings de 10 palabras seleccionadas aleatoriamente del vocabulario. Cada vector de embedding tiene 10 dimensiones y representa la palabra en un espacio semántico. Estos valores en los vectores son los que el modelo ha aprendido para capturar las relaciones de similitud y contexto entre palabras.

Los embeddings generados representan las relaciones de co-ocurrencia en el corpus. Palabras con contextos similares tendrán embeddings cercanos en este espacio. Estos embeddings pueden ser utilizados en tareas de análisis semántico y para medir la similitud de palabras.